# Analisis Simulacion

## Analisis General - Simulación Base

### Pre-procesamiento

In [4]:
import pandas as pd

# 1. Leer los archivos
arma_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARMA_FINAL.xlsx")
arima_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_ARIMA_FINAL.xlsx")
setar_df = pd.read_excel("./datos/Simulacion/Base/resultados_140_SETAR_FINAL.xlsx")

# 2. Asignar la columna ESCENARIO a cada dataframe
arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# 3. Juntarlos uno bajo el otro (Concatenar)
df_total = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Seleccionar solo las columnas requeridas
columnas_deseadas = [
    "Paso", "Config", "Dist", "Var", "Block Bootstrapping", 
    "Sieve Bootstrap", "LSPM", "LSPMW", "AREPD", "MCPS", 
    "AV-MCPS", "DeepAR", "EnCQR-LSTM", "ESCENARIO"
]
df_total = df_total[columnas_deseadas]

# Definimos cuáles son las columnas que representan a los modelos predictivos
modelos = [
    "Block Bootstrapping", "Sieve Bootstrap", "LSPM", "LSPMW", 
    "AREPD", "MCPS", "AV-MCPS", "DeepAR", "EnCQR-LSTM"
]

# 4. Guardar el dataframe consolidado
df_total.to_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx", index=False)

# 5. Generar y mostrar las tablas (Media y Mediana)
metricas = {'MEDIA': 'mean', 'MEDIANA': 'median'}

for nombre_metrica, funcion in metricas.items():
    # Calculamos el valor general según la métrica (mean o median)
    if funcion == 'mean':
        resumen_general = df_total[modelos].mean()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].mean().T
    else:
        resumen_general = df_total[modelos].median()
        resumen_escenarios = df_total.groupby('ESCENARIO')[modelos].median().T

    # Construimos la tabla final para esta métrica
    tabla_resumen = pd.DataFrame(index=modelos)
    tabla_resumen['General'] = resumen_general
    tabla_resumen['ARMA'] = resumen_escenarios['Lineal Estacionario']
    tabla_resumen['ARIMA'] = resumen_escenarios['Lineal No estacionario']
    tabla_resumen['SETAR'] = resumen_escenarios['No lineal Estacionario']

    # Determinar el Mejor_Escenario (valor mínimo entre los tres escenarios)
    escenarios_cols = ['ARMA', 'ARIMA', 'SETAR']
    tabla_resumen['Mejor_Escenario'] = tabla_resumen[escenarios_cols].idxmin(axis=1)

    # Imprimir resultado
    print(f"\n--- Tabla Comparativa de Modelos (Basada en {nombre_metrica}) ---")
    print(tabla_resumen.reset_index().rename(columns={'index': 'Modelo'}).to_string(index=False))


--- Tabla Comparativa de Modelos (Basada en MEDIA) ---
             Modelo  General     ARMA     ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 4.275365 0.946781 11.251601 0.627714           SETAR
    Sieve Bootstrap 0.570491 0.547725  0.547481 0.616268           ARIMA
               LSPM 0.845497 0.811287  1.064804 0.660398           SETAR
              LSPMW 1.572319 0.960743  3.079645 0.676570           SETAR
              AREPD 3.880255 0.936671 10.031183 0.672910           SETAR
               MCPS 1.553139 0.752295  3.218168 0.688955           SETAR
            AV-MCPS 1.578091 0.733188  3.324007 0.677078           SETAR
             DeepAR 1.836024 0.568693  4.329124 0.610255            ARMA
         EnCQR-LSTM 2.675191 1.071557  6.112344 0.841671           SETAR

--- Tabla Comparativa de Modelos (Basada en MEDIANA) ---
             Modelo  General     ARMA    ARIMA    SETAR Mejor_Escenario
Block Bootstrapping 0.989415 0.654169 5.283530 0.540627           SETAR
    Sieve Bo

### Analisis general

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import normaltest
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Simulacion_base")
output_dir.mkdir(parents=True, exist_ok=True)

# Crear carpeta de interacciones
interactions_dir = output_dir / "Interacciones"
interactions_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion/Base/dataframe_consolidado.xlsx")

# 1) CAMBIO DE NOMBRES DE ESCENARIOS
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificar columnas de modelos
var_cols = ['Paso', 'Config', 'Dist', 'Var', 'ESCENARIO']
original_model_cols = [col for col in df.columns if col not in var_cols]

# 2) ORGANIZAR MODELOS POR RENDIMIENTO EN "Lineal No Estacionario (ARIMA)" (Menor a mayor)
target_scenario = "Lineal No Estacionario (ARIMA)"
model_order_scores = df[df['ESCENARIO'] == target_scenario][original_model_cols].mean().sort_values()
model_cols = list(model_order_scores.index)

print("Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):")
for i, m in enumerate(model_cols, 1):
    print(f"{i}. {m}: {model_order_scores[m]:.4f}")

# Mapeo de escenarios
escenarios_map = {
    'Lineal Estacionario (ARMA)': 'Lineal Estacionario (ARMA)',
    'Lineal No Estacionario (ARIMA)': 'Lineal No Estacionario (ARIMA)',
    'No lineal Estacionario (SETAR)': 'No lineal Estacionario (SETAR)'
}

# Definir colores para cada modelo
palette = sns.color_palette("husl", len(model_cols))
model_colors = {model: palette[i] for i, model in enumerate(model_cols)}

# ====================================================================================
# SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: RENDIMIENTO POR ESCENARIOS")
print("="*80)

def plot_performance_by_scenario():
    fig, ax = plt.subplots(figsize=(14, 8))
    
    scenarios = ["Lineal Estacionario (ARMA)", "Lineal No Estacionario (ARIMA)", "No lineal Estacionario (SETAR)"]
    x = np.arange(len(model_cols))
    width = 0.25 
    
    scenario_colors = {
        'Lineal Estacionario (ARMA)': '#5D3FD3',    
        'Lineal No Estacionario (ARIMA)': '#808080', 
        'No lineal Estacionario (SETAR)': '#00A36C'  
    }
    
    for idx, scenario in enumerate(scenarios):
        means = [df[df['ESCENARIO'] == scenario][model].mean() for model in model_cols]
        position = x + (idx - 1) * width
        
        bars = ax.bar(position, means, width, label=scenario, 
                     color=scenario_colors[scenario], alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.2f}', ha='center', va='bottom', fontsize=7, rotation=0)
    
    ax.set_xlabel('Modelo (Ordenados por desempeño en ARIMA)', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title('Rendimiento de Modelos por Escenario (ECRPS)', fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.legend(loc='upper left', ncol=1, fontsize=10, framealpha=0.9)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.1_rendimiento_por_escenario.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.1 guardada")

plot_performance_by_scenario()

def plot_relative_performance():
    base_scenario = 'Lineal Estacionario (ARMA)'
    scenarios_compare = ['Lineal No Estacionario (ARIMA)', 'No lineal Estacionario (SETAR)']
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    y = np.arange(len(model_cols))
    height = 0.35  
    
    for idx, scenario in enumerate(scenarios_compare):
        changes = []
        for model in model_cols:
            base_value = df[df['ESCENARIO'] == base_scenario][model].mean()
            scenario_value = df[df['ESCENARIO'] == scenario][model].mean()
            pct_change = ((scenario_value - base_value) / base_value) * 100
            changes.append(pct_change)
        
        position = y + idx * height
        bars = ax.barh(position, changes, height, label=scenario, alpha=0.85, edgecolor='black', linewidth=0.5)
        
        for bar, val in zip(bars, changes):
            width = bar.get_width()
            ax.text(width + (1 if width > 0 else -1), bar.get_y() + bar.get_height()/2.,
                   f'{val:+.1f}%', ha='left' if val > 0 else 'right', 
                   va='center', fontsize=7)
    
    ax.set_yticks(y + height / 2)
    ax.set_yticklabels(model_cols, fontsize=10)
    ax.set_xlabel('Cambio Relativo (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title(f'Cambio Relativo en ECRPS vs. {base_scenario}', fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1.5)
    ax.legend(loc='best', fontsize=10, framealpha=0.9)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '1.2_cambio_relativo_escenario_base.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 1.2 guardada")

plot_relative_performance()

# ====================================================================================
# SECCIÓN 2: ANÁLISIS POR CONFIG
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: ANÁLISIS POR CONFIG")
print("="*80)

def plot_zscore_heatmap_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Configuración (General)'
    
    pivot_data = data_filtered.groupby('Config')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'2.1{suffix}_zscore_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.1{suffix} guardada")

plot_zscore_heatmap_config()
plot_zscore_heatmap_config('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_config('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_config('No lineal Estacionario (SETAR)', '.c')

def plot_variability_config(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Configuración ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Configuración (General)'
    
    pivot_std = data_filtered.groupby('Config')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    fmt = '.2f' if suffix == '' else '.4f'
    sns.heatmap(pivot_std.T, annot=True, fmt=fmt, cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Configuración', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'2.2{suffix}_variabilidad_config.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 2.2{suffix} guardada")

plot_variability_config()
plot_variability_config('Lineal Estacionario (ARMA)', '.a')
plot_variability_config('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_config('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 3: ANÁLISIS POR DIST
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: ANÁLISIS POR DIST")
print("="*80)

def plot_zscore_heatmap_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Z-scores de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Z-scores de ECRPS por Distribución (General)'
    
    pivot_data = data_filtered.groupby('Dist')[model_cols].mean()
    z_scores = pivot_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(z_scores.T, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                center=0, cbar_kws={'label': 'Z-score'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=11)
    ax.set_ylabel('Modelo', fontsize=11)
    
    plt.tight_layout()
    filename = f'3.1{suffix}_zscore_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.1{suffix} guardada")

plot_zscore_heatmap_dist()
plot_zscore_heatmap_dist('Lineal Estacionario (ARMA)', '.a')
plot_zscore_heatmap_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_zscore_heatmap_dist('No lineal Estacionario (SETAR)', '.c')

def plot_variability_dist(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Distribución ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Distribución (General)'
    
    pivot_std = data_filtered.groupby('Dist')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Distribución', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'3.2{suffix}_variabilidad_dist.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 3.2{suffix} guardada")

plot_variability_dist()
plot_variability_dist('Lineal Estacionario (ARMA)', '.a')
plot_variability_dist('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_dist('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 4: ANÁLISIS POR VAR
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: ANÁLISIS POR VAR")
print("="*80)

def plot_evolution_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Varianza (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    var_values = sorted(data_filtered['Var'].unique())
    
    for model in model_cols:
        means = []
        for var in var_values:
            mean_val = data_filtered[data_filtered['Var'] == var][model].mean()
            means.append(mean_val)
        
        ax.plot(var_values, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'4.1{suffix}_evolucion_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.1{suffix} guardada")

plot_evolution_var()
plot_evolution_var('Lineal Estacionario (ARMA)', '.a')
plot_evolution_var('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_var('No lineal Estacionario (SETAR)', '.c')

def plot_variability_var(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Varianza ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Varianza (General)'
    
    pivot_std = data_filtered.groupby('Var')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Varianza', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'4.2{suffix}_variabilidad_var.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 4.2{suffix} guardada")

plot_variability_var()
plot_variability_var('Lineal Estacionario (ARMA)', '.a')
plot_variability_var('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_var('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5: ANÁLISIS POR PASO (HORIZONTE)")
print("="*80)

def plot_evolution_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Evolución de ECRPS por Horizonte de Pronóstico ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Evolución de ECRPS por Horizonte de Pronóstico (General)'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    pasos = sorted(data_filtered['Paso'].unique())
    
    for model in model_cols:
        means = []
        for paso in pasos:
            mean_val = data_filtered[data_filtered['Paso'] == paso][model].mean()
            means.append(mean_val)
        
        ax.plot(pasos, means, marker='o', label=model, color=model_colors[model],
                linewidth=2.5, markersize=7, alpha=0.85)
    
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=9, ncol=2, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    filename = f'5.1{suffix}_evolucion_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.1{suffix} guardada")

plot_evolution_paso()
plot_evolution_paso('Lineal Estacionario (ARMA)', '.a')
plot_evolution_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_evolution_paso('No lineal Estacionario (SETAR)', '.c')

def plot_variability_paso(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Desv. Estándar de ECRPS por Horizonte ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Desv. Estándar de ECRPS por Horizonte (General)'
    
    pivot_std = data_filtered.groupby('Paso')[model_cols].std()
    
    fig, ax = plt.subplots(figsize=(14, 8))
    sns.heatmap(pivot_std.T, annot=True, fmt='.4f', cmap='YlOrRd', 
                cbar_kws={'label': 'Desv. Estándar'}, ax=ax,
                linewidths=0.5, linecolor='gray')
    
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Horizonte de Pronóstico', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = f'5.2{suffix}_variabilidad_paso.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 5.2{suffix} guardada")

plot_variability_paso()
plot_variability_paso('Lineal Estacionario (ARMA)', '.a')
plot_variability_paso('Lineal No Estacionario (ARIMA)', '.b')
plot_variability_paso('No lineal Estacionario (SETAR)', '.c')

# ====================================================================================
# SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5.5: ANÁLISIS DE INTERACCIONES")
print("="*80)

# Función auxiliar para configurar el grid de modelos
def get_model_grid_axes(n_models):
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
    return fig, axes.flatten(), n_rows, n_cols

# INTERACCIÓN 3 (Lista 8): DIST × PASO - Subplots por Modelo
def plot_interaction_dist_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, n_rows, n_cols = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_dist = sns.color_palette("viridis", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='o', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.7{suffix}_interaccion_dist_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 8 (5.7) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 4 (Lista 9): DIST × VAR - Subplots por Modelo
def plot_interaction_dist_var(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Dist × Var por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    dists = sorted(data_filtered['Dist'].unique())
    vars_val = sorted(data_filtered['Var'].unique())
    colors_dist = sns.color_palette("magma", len(dists))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for d_idx, dist in enumerate(dists):
            means = [data_filtered[(data_filtered['Dist'] == dist) & (data_filtered['Var'] == v)][model].mean() for v in vars_val]
            ax.plot(vars_val, means, marker='s', label=dist, color=colors_dist[d_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Varianza')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Distribución", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.8{suffix}_interaccion_dist_var_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 9 (5.8) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 5 (Lista 10): CONFIG × PASO - Subplots por Modelo
def plot_interaction_config_paso(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Config × Paso por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    configs = sorted(data_filtered['Config'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_conf = sns.color_palette("tab10", len(configs))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for c_idx, config in enumerate(configs):
            means = [data_filtered[(data_filtered['Config'] == config) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='^', label=config, color=colors_conf[c_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Config", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.9{suffix}_interaccion_config_paso_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 10 (5.9) {suffix} guardada: Subplots por Modelo")

# INTERACCIÓN 6 (Lista 11): VAR × HORIZONTE - Subplots por Modelo
def plot_interaction_var_horizonte(scenario=None, suffix=''):
    data_filtered = df[df['ESCENARIO'] == scenario].copy() if scenario else df.copy()
    title = f'Interacción Var × Horizonte por Modelo ({"General" if not scenario else scenario})'
    
    fig, axes, _, _ = get_model_grid_axes(len(model_cols))
    vars_val = sorted(data_filtered['Var'].unique())
    pasos = sorted(data_filtered['Paso'].unique())
    colors_var = sns.color_palette("rocket", len(vars_val))

    for idx, model in enumerate(model_cols):
        ax = axes[idx]
        for v_idx, var in enumerate(vars_val):
            means = [data_filtered[(data_filtered['Var'] == var) & (data_filtered['Paso'] == p)][model].mean() for p in pasos]
            ax.plot(pasos, means, marker='d', label=f'Var {var}', color=colors_var[v_idx], linewidth=2)
        
        ax.set_title(f'Modelo: {model}', fontweight='bold')
        ax.set_xlabel('Horizonte (Paso)')
        ax.set_ylabel('ECRPS Promedio')
        ax.grid(True, alpha=0.3)
        if idx == 0: ax.legend(title="Varianza", fontsize=8)

    for i in range(len(model_cols), len(axes)): axes[i].set_visible(False)
    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(interactions_dir / f'5.10{suffix}_interaccion_var_horizonte_modelos.png', bbox_inches='tight')
    plt.close()
    print(f"✓ Gráfica 11 (5.10) {suffix} guardada: Subplots por Modelo")


# ====================================================================================
# EJECUCIÓN DE LAS NUEVAS FUNCIONES
# ====================================================================================

for sc_name, sc_suf in [ (None, ''), ('Lineal Estacionario (ARMA)', '.a'), 
                        ('Lineal No Estacionario (ARIMA)', '.b'), 
                        ('No lineal Estacionario (SETAR)', '.c') ]:
    plot_interaction_dist_paso(sc_name, sc_suf)
    plot_interaction_dist_var(sc_name, sc_suf)
    plot_interaction_config_paso(sc_name, sc_suf)
    plot_interaction_var_horizonte(sc_name, sc_suf)

# ====================================================================================
# SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6: ROBUSTEZ Y TEST DIEBOLD-MARIANO")
print("="*80)

def plot_robustness():
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cv_data = []
    for model in model_cols:
        cv = df[model].std() / df[model].mean()
        cv_data.append((model, cv))
    
    cv_df = pd.DataFrame(cv_data, columns=['Modelo', 'CV'])
    
    # CAMBIO 1: Ordenar de menor a mayor CV para coherencia
    cv_df = cv_df.sort_values('CV')
    
    colors_cv = ['#2ecc71' if cv < cv_df['CV'].median() else '#e74c3c' 
                 for cv in cv_df['CV']]
    
    bars = ax.barh(cv_df['Modelo'], cv_df['CV'], color=colors_cv, alpha=0.8, edgecolor='black')
    
    for bar, cv in zip(bars, cv_df['CV']):
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2.,
               f'{cv:.4f}', ha='left', va='center', fontsize=9)
    
    ax.set_xlabel('Coeficiente de Variación', fontsize=12, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
    ax.set_title('Robustez: Coeficiente de Variación\n(Ordenado de menor a mayor - Menor valor indica mayor estabilidad)', 
                  fontsize=14, fontweight='bold', pad=20)
    ax.axvline(x=cv_df['CV'].median(), color='black', linestyle='--', linewidth=1.5, alpha=0.5, label='Mediana')
    ax.legend(loc='best', fontsize=10)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.savefig(output_dir / '6.1_robustez_coeficiente_variacion.png', bbox_inches='tight')
    plt.close()
    print("✓ Gráfica 6.1 guardada (ordenada de menor a mayor CV)")

plot_robustness()

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano modificado con corrección Harvey-Leybourne-Newbold (1997)
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hlm_dm_stat : float
        Estadístico DM corregido (HLN-DM)
    p_value : float
        P-valor usando distribución t-Student con T-1 grados de libertad
    dm_stat : float
        Estadístico DM original (sin corrección)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    # Calcular autocovarianzas
    def gamma_d(k):
        if k == 0:
            return np.var(d, ddof=1)
        else:
            return np.mean((d[k:] - d_bar) * (d[:-k] - d_bar))
    
    # Estimar la varianza de largo plazo usando Newey-West
    # Para h-step-ahead forecasts, incluimos hasta h-1 lags
    gamma_0 = gamma_d(0)
    gamma_sum = gamma_0
    
    if h > 1:
        for k in range(1, h):
            gamma_k = gamma_d(k)
            gamma_sum += 2 * gamma_k
    
    var_d = gamma_sum / T
    
    if var_d <= 0:
        return 0, 1.0, 0
    
    # Estadístico DM original
    dm_stat = d_bar / np.sqrt(var_d)
    
    # Corrección Harvey-Leybourne-Newbold (1997)
    correction_factor = np.sqrt((T + 1 - 2*h + h*(h-1)) / T)
    hln_dm_stat = correction_factor * dm_stat
    
    # P-valor usando t-Student con T-1 grados de libertad
    df = T - 1
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

def plot_dm_test_heatmap(scenario=None, suffix=''):
    if scenario:
        data_filtered = df[df['ESCENARIO'] == scenario].copy()
        title = f'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni ({scenario})'
    else:
        data_filtered = df.copy()
        title = 'Test Diebold-Mariano Modificado (HLN-DM)\ncon Corrección de Bonferroni (General)'
    
    # Determinar el horizonte de pronóstico promedio
    h_forecast = int(data_filtered['Paso'].mean())
    
    n_models = len(model_cols)
    n_comparisons = n_models * (n_models - 1) / 2
    alpha = 0.05
    bonferroni_alpha = alpha / n_comparisons
    
    results_matrix = np.zeros((n_models, n_models))
    p_values = np.zeros((n_models, n_models))
    dm_stats = np.zeros((n_models, n_models))
    
    for i, model1 in enumerate(model_cols):
        for j, model2 in enumerate(model_cols):
            if i == j:
                results_matrix[i, j] = 0  
                p_values[i, j] = 1.0
                dm_stats[i, j] = 0
            elif i < j:
                errors1 = data_filtered[model1].values
                errors2 = data_filtered[model2].values
                hln_dm_stat, p_val, dm_original = modified_diebold_mariano_test(
                    errors1, errors2, h=h_forecast
                )
                
                p_values[i, j] = p_val
                p_values[j, i] = p_val
                dm_stats[i, j] = hln_dm_stat
                dm_stats[j, i] = -hln_dm_stat
                
                if p_val < bonferroni_alpha:
                    mean1 = np.mean(errors1)
                    mean2 = np.mean(errors2)
                    if mean1 < mean2:  
                        results_matrix[i, j] = 1  
                        results_matrix[j, i] = -1  
                    else:
                        results_matrix[i, j] = -1
                        results_matrix[j, i] = 1
                else:
                    results_matrix[i, j] = 0
                    results_matrix[j, i] = 0
    
    fig, ax = plt.subplots(figsize=(14, 11))
    cmap = plt.cm.colors.ListedColormap(['#e74c3c', '#fff9c4', '#2ecc71'])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    
    im = ax.imshow(results_matrix, cmap=cmap, norm=norm, aspect='auto')
    ax.set_xticks(np.arange(n_models))
    ax.set_yticks(np.arange(n_models))
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.set_yticklabels(model_cols, fontsize=9)
    
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                text = '-'
                color = 'black'
            else:
                val = results_matrix[i, j]
                p_val = p_values[i, j]
                dm_val = dm_stats[i, j]
                if val == 1:
                    text = f'✓\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                elif val == -1:
                    text = f'✗\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                else:
                    text = f'≈\nHLN-DM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'black'
            ax.text(j, i, text, ha='center', va='center', 
                   color=color, fontsize=6, fontweight='bold')
    
    # Obtener T para el título
    T = len(data_filtered)
    df_test = T - 1
    
    ax.set_title(title + f'\n(h={h_forecast}, T={T}, df={df_test}, α ajustado={bonferroni_alpha:.5f})', 
                 fontsize=11, fontweight='bold', pad=20)
    ax.set_xlabel('Modelo (Columna)', fontsize=11)
    ax.set_ylabel('Modelo (Fila)', fontsize=11)
    
    legend_elements = [
        plt.Rectangle((0,0),1,1, facecolor='#2ecc71', label='Fila supera columna (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#e74c3c', label='Columna supera fila (p < α)'),
        plt.Rectangle((0,0),1,1, facecolor='#fff9c4', label='Sin diferencia significativa (p ≥ α)')
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=9)
    
    # Agregar nota metodológica
    note_text = ('Nota: Se utiliza el test HLN-DM con corrección para muestras finitas.\n'
                 'Distribución: t-Student. Ajuste: Bonferroni para comparaciones múltiples.')
    plt.figtext(0.5, -0.02, note_text, ha='center', fontsize=8, style='italic', 
                wrap=True, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    filename = f'6.2{suffix}_dm_test_hln_bonferroni.png'
    plt.savefig(output_dir / filename, bbox_inches='tight')
    plt.close()
    
    # Guardar resultados en Excel
    results_df = pd.DataFrame(results_matrix, index=model_cols, columns=model_cols)
    pvalues_df = pd.DataFrame(p_values, index=model_cols, columns=model_cols)
    dmstats_df = pd.DataFrame(dm_stats, index=model_cols, columns=model_cols)
    
    summary_data = []
    for model in model_cols:
        idx = model_cols.index(model)
        victorias = int(np.sum(results_matrix[idx, :] == 1))
        derrotas = int(np.sum(results_matrix[idx, :] == -1))
        empates = int(np.sum(results_matrix[idx, :] == 0)) - 1
        mean_ecrps = data_filtered[model].mean()
        std_ecrps = data_filtered[model].std()
        cv_ecrps = std_ecrps / mean_ecrps
        summary_data.append({
            'Modelo': model,
            'Victorias': victorias,
            'Derrotas': derrotas,
            'Empates': empates,
            'Tasa_Victoria': f"{(victorias / (n_models - 1)) * 100:.1f}%",
            'ECRPS_Promedio': f"{mean_ecrps:.4f}",
            'ECRPS_Desv_Std': f"{std_ecrps:.4f}",
            'Coef_Variacion': f"{cv_ecrps:.4f}"
        })
    summary_df = pd.DataFrame(summary_data)
    
    # Información metodológica
    method_info = pd.DataFrame({
        'Parámetro': ['Horizonte de pronóstico (h)', 'Tamaño muestra (T)', 
                      'Grados libertad (df)', 'Alpha nominal', 
                      'Alpha ajustado (Bonferroni)', 'Número comparaciones',
                      'Distribución', 'Corrección aplicada'],
        'Valor': [h_forecast, T, df_test, alpha, bonferroni_alpha, 
                  int(n_comparisons), 't-Student', 'Harvey-Leybourne-Newbold (1997)']
    })
    
    with pd.ExcelWriter(output_dir / f'6.2{suffix}_dm_test_hln_resultados.xlsx') as writer:
        method_info.to_excel(writer, sheet_name='Metodologia', index=False)
        results_df.to_excel(writer, sheet_name='Matriz_Resultados')
        pvalues_df.to_excel(writer, sheet_name='P_valores')
        dmstats_df.to_excel(writer, sheet_name='Estadisticos_HLN_DM')
        summary_df.to_excel(writer, sheet_name='Resumen_Modelos', index=False)
    
    print(f"✓ Gráfica 6.2{suffix} guardada (Test HLN-DM con h={h_forecast}, T={T}, df={df_test})")

plot_dm_test_heatmap()
plot_dm_test_heatmap('Lineal Estacionario (ARMA)', '.a')
plot_dm_test_heatmap('Lineal No Estacionario (ARIMA)', '.b')
plot_dm_test_heatmap('No lineal Estacionario (SETAR)', '.c')

print("\n" + "="*80)
print("PROCESO COMPLETADO")
print("="*80)
print("\nMejoras implementadas:")
print("1. ✓ Orden consistente en gráficas 1.1 y 1.2")
print("2. ✓ Formato de 2 decimales en heatmap 2.2 general")
print("3. ✓ Solo Coeficiente de Variación en gráfica 6.1 (ordenado menor a mayor)")
print("4. ✓ Test Diebold-Mariano con corrección HLN y distribución t-Student")
print("5. ✓ Horizonte de pronóstico (h) incorporado en el análisis")
print("\n" + "="*80)
print("ANÁLISIS DE INTERACCIONES (Carpeta: Interacciones/)")
print("="*80)
print("6. ✓ Config × Var: Heatmaps por modelo (matriz 3×3)")
print("7. ✓ Config × Dist: Heatmaps por modelo (matriz 3×3)")
print("8. ✓ Dist × Paso: Gráficas de líneas por distribución")
print("9. ✓ Dist × Var: Gráficas de líneas por distribución")
print("10. ✓ Config × Paso: Gráficas de líneas por configuración")
print("11. ✓ Var × Horizonte: Gráficas de líneas por varianza")
print("="*80)
print(f"\nTotal de gráficas generadas en Interacciones/: {6 * 4} archivos")
print("(6 tipos de interacción × 4 escenarios: General + 3 específicos)")
print("="*80)

Nuevo orden de modelos (basado en Lineal No Estacionario (ARIMA)):
1. Sieve Bootstrap: 0.5475
2. LSPM: 1.0648
3. LSPMW: 3.0796
4. MCPS: 3.2182
5. AV-MCPS: 3.3240
6. DeepAR: 4.3291
7. EnCQR-LSTM: 6.1123
8. AREPD: 10.0312
9. Block Bootstrapping: 11.2516

SECCIÓN 1: RENDIMIENTO POR ESCENARIOS
✓ Gráfica 1.1 guardada
✓ Gráfica 1.2 guardada

SECCIÓN 2: ANÁLISIS POR CONFIG
✓ Gráfica 2.1 guardada
✓ Gráfica 2.1.a guardada
✓ Gráfica 2.1.b guardada
✓ Gráfica 2.1.c guardada
✓ Gráfica 2.2 guardada
✓ Gráfica 2.2.a guardada
✓ Gráfica 2.2.b guardada
✓ Gráfica 2.2.c guardada

SECCIÓN 3: ANÁLISIS POR DIST
✓ Gráfica 3.1 guardada
✓ Gráfica 3.1.a guardada
✓ Gráfica 3.1.b guardada
✓ Gráfica 3.1.c guardada
✓ Gráfica 3.2 guardada
✓ Gráfica 3.2.a guardada
✓ Gráfica 3.2.b guardada
✓ Gráfica 3.2.c guardada

SECCIÓN 4: ANÁLISIS POR VAR
✓ Gráfica 4.1 guardada
✓ Gráfica 4.1.a guardada
✓ Gráfica 4.1.b guardada
✓ Gráfica 4.1.c guardada
✓ Gráfica 4.2 guardada
✓ Gráfica 4.2.a guardada
✓ Gráfica 4.2.b guardada
✓ Gráfica

## Analisis Diferenciado

### Pre-procesamiento

In [23]:
import pandas as pd
import numpy as np

# 1. Cargar los archivos
path_diff = "./datos/Simulacion/Diferenciado/resultados_140_ARIMA_CON_DIFERENCIACION.xlsx"
path_no_diff = "./datos/Simulacion/Diferenciado/resultados_140_ARIMA_FINAL.xlsx"

arima_Diff_df = pd.read_excel(path_diff)
arima_df = pd.read_excel(path_no_diff)

# 2. Agregar la columna 'Diferenciacion' al dataframe que no la tiene
arima_df['Diferenciacion'] = 'No'

# 3. Unir los dataframes (uno debajo del otro)
# El orden de las columnas se ajustará automáticamente
df_unido = pd.concat([arima_Diff_df, arima_df], ignore_index=True)

# 4. Guardar la unión en la misma carpeta
path_salida = "./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx"
df_unido.to_excel(path_salida, index=False)

# 5. Crear la tabla comparativa (Resumen)
# Definimos las métricas/modelos a comparar
modelos_metricas = [
    'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
    'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap'
]

# Calculamos el promedio para cada caso
resumen_no_diff = arima_df[modelos_metricas].mean()
resumen_diff = arima_Diff_df[modelos_metricas].mean()

# Construir el DataFrame comparativo
tabla_comparativa = pd.DataFrame({
    'ARIMA': resumen_no_diff,
    'ARIMA_Diff': resumen_diff
})

# Determinar el mejor escenario (el que tenga el valor menor, asumiendo que son errores)
tabla_comparativa['Mejor_Escenario'] = np.where(
    tabla_comparativa['ARIMA'] < tabla_comparativa['ARIMA_Diff'], 
    'ARIMA', 
    'ARIMA_Diff'
)

# Formatear la tabla para que se vea limpia
tabla_comparativa.index.name = 'Modelo'
tabla_comparativa = tabla_comparativa.sort_index()

# Mostrar resultados
print("Tabla Comparativa de Desempeño (Promedios):")
print(tabla_comparativa.to_string())

print(f"\nArchivo de unión guardado en: {path_salida}")

Tabla Comparativa de Desempeño (Promedios):
                         ARIMA  ARIMA_Diff Mejor_Escenario
Modelo                                                    
AREPD                10.031183    0.704149      ARIMA_Diff
AV-MCPS               3.324007    0.654257      ARIMA_Diff
Block Bootstrapping  11.251601    0.666133      ARIMA_Diff
DeepAR                4.329124    0.561822      ARIMA_Diff
EnCQR-LSTM            6.112344    0.880288      ARIMA_Diff
LSPM                  1.064804    0.648039      ARIMA_Diff
LSPMW                 3.079645    0.767172      ARIMA_Diff
MCPS                  3.218168    0.677471      ARIMA_Diff
Sieve Bootstrap       0.547481    0.546020      ARIMA_Diff

Archivo de unión guardado en: ./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx


### Analisis general

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# 1. PREPARACIÓN DE DIRECTORIOS
output_dir = Path("./Resultados_analisis/Multi_d")
output_dir.mkdir(parents=True, exist_ok=True)
plots_dir = output_dir / "Graficos_Analisis"
plots_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
path_excel = "./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
try:
    df = pd.read_excel(path_excel)
    print(f"✓ Datos cargados: {df.shape}")
    print(f"✓ Columnas: {df.columns.tolist()}")
except Exception as e:
    print(f"Error crítico: {e}")
    exit()

# Identificar columnas
var_cols = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 
            'Varianza', 'Modalidad', 'Valor_Observado']
model_cols = [col for col in df.columns if col not in var_cols]

print(f"✓ Modelos identificados: {model_cols}")
print(f"✓ Valores únicos de d: {sorted(df['d'].unique())}")

# ====================================================================================
# SECCIÓN 1: COMPARACIÓN INDIVIDUAL POR VALOR DE d
# ====================================================================================
def plot_individual_comparisons():
    print("\n=== Generando gráficos de barras con valores por d ===")
    
    # Para cada valor de d
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        stats_list = []
        subset_d = df[df['d'] == d_val]
        
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            stats_list.append({'Modelo': model, 'Sin': sin, 'Con': con, 'Mejora': mejora})
        
        df_stats = pd.DataFrame(stats_list).sort_values(by='Mejora', ascending=False)
        
        # 1.1 Barras Comparativas por d
        fig, ax = plt.subplots(figsize=(14, 6))
        x = np.arange(len(df_stats))
        width = 0.35
        
        b1 = ax.bar(x - width/2, df_stats['Sin'], width, label='Sin Dif.', 
                    color='#e74c3c', edgecolor='black', alpha=0.8)
        b2 = ax.bar(x + width/2, df_stats['Con'], width, label='Con Dif.', 
                    color='#2ecc71', edgecolor='black', alpha=0.8)
        
        ax.bar_label(b1, padding=3, fmt='%.3f', fontsize=7, rotation=45)
        ax.bar_label(b2, padding=3, fmt='%.3f', fontsize=7, rotation=45)
        
        ax.set_title(f'ECRPS Promedio por Modelo (d={d_val})', fontweight='bold', fontsize=12)
        ax.set_xticks(x)
        ax.set_xticklabels(df_stats['Modelo'], rotation=45, ha='right')
        ax.set_ylabel('ECRPS')
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(plots_dir / f'1.1_barras_ecrps_d{d_val}.png')
        plt.close()

# ====================================================================================
# SECCIÓN 2: HEATMAP d vs MODELO
# ====================================================================================
def plot_heatmap_d_vs_modelo():
    print("\n=== Generando Heatmap d vs Modelo ===")
    
    results = []
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            results.append({'d': d_val, 'Modelo': model, 'Mejora_%': mejora})
    
    df_mejora = pd.DataFrame(results)
    pivot = df_mejora.pivot(index='Modelo', columns='d', values='Mejora_%')
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0, 
                cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
    plt.title("Mejora Porcentual (%): Modelo vs Valor de d", fontweight='bold', fontsize=14)
    plt.xlabel("Valor de d (Grado de diferenciación)", fontweight='bold')
    plt.ylabel("Modelo", fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / '2_heatmap_d_vs_modelo.png')
    plt.close()
    
    # Guardar tabla
    pivot.to_excel(output_dir / '2_tabla_d_vs_modelo.xlsx')

# ====================================================================================
# SECCIÓN 3: HEATMAP SIGNIFICANCIA (d vs Diferenciación)
# ====================================================================================
def plot_heatmap_significancia():
    print("\n=== Generando Heatmap de Significancia d vs Diferenciación ===")
    
    results = []
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].values
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].values
            
            if len(sin) > 1 and len(con) > 1:
                stat, p_valor = stats.ttest_ind(sin, con)
                significativo = 1 if p_valor < 0.05 else 0
                # Determinar dirección
                if p_valor < 0.05:
                    if sin.mean() > con.mean():
                        valor = 1  # Diferenciación es mejor
                    else:
                        valor = -1  # Sin diferenciación es mejor
                else:
                    valor = 0  # No significativo
            else:
                valor = 0
            
            results.append({'d': d_val, 'Modelo': model, 'Significancia': valor})
    
    df_sig = pd.DataFrame(results)
    pivot_sig = df_sig.pivot(index='Modelo', columns='d', values='Significancia')
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot_sig, annot=True, fmt=".0f", cmap="RdYlGn", center=0,
                cbar_kws={'label': 'Significancia (-1: Sin Dif mejor | 0: No sig. | 1: Con Dif mejor)'},
                linewidths=0.5, vmin=-1, vmax=1)
    plt.title("Significancia Estadística: d vs Diferenciación por Modelo", fontweight='bold', fontsize=14)
    plt.xlabel("Valor de d", fontweight='bold')
    plt.ylabel("Modelo", fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / '3_heatmap_significancia_d.png')
    plt.close()
    
    pivot_sig.to_excel(output_dir / '3_tabla_significancia_d.xlsx')

# ====================================================================================
# SECCIÓN 4: INTERACCIONES d vs Distribución, Config y Varianza
# ====================================================================================
def plot_interacciones_d():
    print("\n=== Generando Interacciones d vs Dist, Config y Varianza ===")
    
    # 4.1 d vs Distribución
    def get_improvement_d_group(group_col):
        results = []
        d_values = sorted(df['d'].unique())
        for d_val in d_values:
            subset_d = df[df['d'] == d_val]
            for val in subset_d[group_col].unique():
                subset = subset_d[subset_d[group_col] == val]
                for model in model_cols:
                    sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                    con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                    mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
                    results.append({'d': d_val, group_col: val, 'Modelo': model, 'Mejora': mejora})
        return pd.DataFrame(results)
    
    # 4.1 Heatmap d vs Distribución (agrupado por modelo)
    df_dist = get_improvement_d_group('Distribución')
    
    for model in model_cols:
        subset = df_dist[df_dist['Modelo'] == model]
        pivot = subset.pivot(index='d', columns='Distribución', values='Mejora')
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0)
        plt.title(f"Mejora (%) d vs Distribución - {model}", fontweight='bold')
        plt.xlabel("Distribución")
        plt.ylabel("Valor de d")
        plt.tight_layout()
        plt.savefig(plots_dir / f'4.1_heatmap_d_dist_{model}.png')
        plt.close()
    
    # 4.2 Heatmap d vs Varianza (líneas por modelo)
    df_var = get_improvement_d_group('Varianza')
    
    plt.figure(figsize=(14, 8))
    for model in model_cols:
        subset = df_var[df_var['Modelo'] == model]
        grouped = subset.groupby(['d', 'Varianza'])['Mejora'].mean().reset_index()
        for var_val in grouped['Varianza'].unique():
            data = grouped[grouped['Varianza'] == var_val]
            plt.plot(data['d'], data['Mejora'], marker='o', label=f'{model}-Var{var_val}', alpha=0.7)
    
    plt.axhline(0, color='black', linestyle='--', alpha=0.5)
    plt.title("Tendencia Mejora (%): d vs Varianza por Modelo", fontweight='bold')
    plt.xlabel("Valor de d")
    plt.ylabel("Mejora Porcentual")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(plots_dir / '4.2_linea_d_varianza.png')
    plt.close()
    
    # 4.3 Heatmap d vs Config (por modelo)
    # Nota: Asumiendo que Config se deriva de (p, q) o ARMA_base
    # Si no existe, usar ARMA_base como proxy
    if 'ARMA_base' in df.columns:
        df_config = get_improvement_d_group('ARMA_base')
        
        for model in model_cols:
            subset = df_config[df_config['Modelo'] == model]
            pivot = subset.pivot(index='d', columns='ARMA_base', values='Mejora')
            
            plt.figure(figsize=(12, 8))
            sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0)
            plt.title(f"Mejora (%) d vs Configuración Base - {model}", fontweight='bold')
            plt.xlabel("Configuración ARMA")
            plt.ylabel("Valor de d")
            plt.tight_layout()
            plt.savefig(plots_dir / f'4.3_heatmap_d_config_{model}.png')
            plt.close()

# ====================================================================================
# SECCIÓN 6: TABLA DM POR VALOR DE d
# ====================================================================================
def run_dm_analysis():
    print("\n=== Calculando Tabla DM por valor de d ===")
    
    d_values = sorted(df['d'].unique())
    all_results = []
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].values
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].values
            
            if len(sin) > 1 and len(con) > 1:
                mean_sin, mean_con = sin.mean(), con.mean()
                mejora_pct = ((mean_sin - mean_con) / mean_sin) * 100 if mean_sin != 0 else 0
                
                stat, p_valor = stats.ttest_ind(sin, con)
                significativo = 'Sí' if p_valor < 0.05 else 'No'
                
                if p_valor < 0.05:
                    conclusion = "Diferenciación mejora" if mejora_pct > 0 else "Sin dif mejor"
                else:
                    conclusion = "Sin diferencia significativa"
                
                all_results.append({
                    'd': d_val, 'Modelo': model, 
                    'ECRPS_Sin_Diff': mean_sin, 'ECRPS_Con_Diff': mean_con,
                    'Mejora_%': mejora_pct, 'p_valor': p_valor, 
                    'Significativo': significativo, 'Conclusion': conclusion
                })
    
    dm_df = pd.DataFrame(all_results).sort_values(by=['d', 'Mejora_%'], ascending=[True, False])
    dm_df.to_excel(output_dir / '6_tabla_dm_por_d.xlsx', index=False)

# ====================================================================================
# SECCIÓN 9: EXCEL DETALLADO POR d (Agrupado, Ordenado)
# ====================================================================================
def generate_detailed_excel():
    print("\n=== Generando Excel detallado por d (ordenado por mejora) ===")
    file_path = output_dir / "9_Analisis_Detallado_Modelos_por_d.xlsx"
    
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        for model in model_cols:
            # Agrupar por d, ARMA_base, Distribución, Varianza (promediando Pasos)
            temp = df.groupby(['d', 'ARMA_base', 'Distribución', 'Varianza', 'Modalidad'])[model].mean().reset_index()
            
            # Pivotar
            detailed = temp.pivot_table(
                index=['d', 'ARMA_base', 'Distribución', 'Varianza'],
                columns='Modalidad',
                values=model
            ).reset_index()
            
            detailed.columns.name = None
            detailed = detailed.rename(columns={
                'SIN_DIFF': 'ECRPS_Sin_Dif', 
                'CON_DIFF': 'ECRPS_Con_Dif'
            })
            
            detailed['Mejora_Absoluta'] = detailed['ECRPS_Sin_Dif'] - detailed['ECRPS_Con_Dif']
            detailed['Mejora_%'] = (detailed['Mejora_Absoluta'] / detailed['ECRPS_Sin_Dif']) * 100
            
            # Ordenar por d y luego por mejora
            detailed = detailed.sort_values(by=['d', 'Mejora_%'], ascending=[True, False])
            
            sheet_name = model[:31]
            detailed.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Formatos condicionales
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            fmt_green = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
            fmt_red = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
            
            # Aplicar a columna Mejora_%
            col_idx = detailed.columns.get_loc('Mejora_%')
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx, {
                'type': 'cell', 'criteria': '>', 'value': 0, 'format': fmt_green
            })
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx, {
                'type': 'cell', 'criteria': '<', 'value': 0, 'format': fmt_red
            })

# ====================================================================================
# SECCIÓN ADICIONAL: GRÁFICOS PREVIOS MANTENIDOS
# ====================================================================================
def plot_mejora_general():
    """Gráfico general de mejora promedio por modelo (todos los d)"""
    print("\n=== Generando gráfico general de mejora ===")
    
    stats_list = []
    for model in model_cols:
        sin = df[df['Modalidad'] == 'SIN_DIFF'][model].mean()
        con = df[df['Modalidad'] == 'CON_DIFF'][model].mean()
        mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
        stats_list.append({'Modelo': model, 'Mejora_%': mejora})
    
    df_stats = pd.DataFrame(stats_list).sort_values(by='Mejora_%', ascending=False)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    colors = ['#2ecc71' if m > 0 else '#e74c3c' for m in df_stats['Mejora_%']]
    bars = ax.barh(df_stats['Modelo'], df_stats['Mejora_%'], color=colors, edgecolor='black')
    ax.bar_label(bars, padding=5, fmt='%.1f%%', fontweight='bold')
    ax.axvline(0, color='black', lw=1)
    ax.set_title('Mejora Porcentual Global con Diferenciación (Todos los d)', fontweight='bold')
    ax.set_xlabel('Mejora (%)')
    plt.tight_layout()
    plt.savefig(plots_dir / '0_mejora_global.png')
    plt.close()

# ====================================================================================
# EJECUCIÓN PRINCIPAL
# ====================================================================================
if __name__ == "__main__":
    print("\n" + "="*80)
    print("INICIANDO ANÁLISIS MULTI-D")
    print("="*80)
    
    plot_mejora_general()
    plot_individual_comparisons()
    plot_heatmap_d_vs_modelo()
    plot_heatmap_significancia()
    plot_interacciones_d()
    run_dm_analysis()
    generate_detailed_excel()
    
    print("\n" + "="*80)
    print(f"✓ ANÁLISIS COMPLETADO EXITOSAMENTE")
    print(f"✓ Resultados guardados en: {output_dir}")
    print(f"✓ Gráficos guardados en: {plots_dir}")
    print("="*80)

✓ Datos cargados: (3360, 14)
Generando gráficos de barras con valores...
Generando Heatmaps y Gráfico de Líneas...
Calculando tabla DM...
Generando Excel detallado (Sin Paso, ordenado por mejora)...

ANÁLISIS COMPLETADO. Carpeta: Resultados_analisis\Simulacion_diff


## Aumento d ARIMA

### Pre-procesamiento

In [14]:
import pandas as pd
import numpy as np
from scipy import stats

# Cargar datos
base = pd.read_excel("./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx")

# Seleccionar columnas necesarias
columnas_seleccionadas = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 
                          'Distribución', 'Varianza', 'Modalidad', 'Sieve Bootstrap']
datos = base[columnas_seleccionadas].copy()

# Renombrar columna para claridad
datos.rename(columns={'Sieve Bootstrap': 'ECRPS'}, inplace=True)

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano modificado con corrección Harvey-Leybourne-Newbold (1997)
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico DM corregido (HLN-DM)
    p_value : float
        P-valor usando distribución t-Student con T-1 grados de libertad
    dm_stat : float
        Estadístico DM original (sin corrección)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    # Calcular autocovarianzas
    def gamma_d(k):
        if k == 0:
            return np.var(d, ddof=1)
        else:
            return np.mean((d[k:] - d_bar) * (d[:-k] - d_bar))
    
    # Estimar la varianza de largo plazo usando Newey-West
    # Para h-step-ahead forecasts, incluimos hasta h-1 lags
    gamma_0 = gamma_d(0)
    gamma_sum = gamma_0
    
    if h > 1:
        for k in range(1, h):
            gamma_k = gamma_d(k)
            gamma_sum += 2 * gamma_k
    
    var_d = gamma_sum / T
    
    if var_d <= 0:
        return 0, 1.0, 0
    
    # Estadístico DM original
    dm_stat = d_bar / np.sqrt(var_d)
    
    # Corrección Harvey-Leybourne-Newbold (1997)
    correction_factor = np.sqrt((T + 1 - 2*h + h*(h-1)) / T)
    hln_dm_stat = correction_factor * dm_stat
    
    # P-valor usando t-Student con T-1 grados de libertad
    df = T - 1
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

# Obtener valores únicos de d
valores_d = sorted(datos['d'].unique())

# Lista para almacenar resultados
resultados = []

# Iterar sobre cada valor de d
for d_val in valores_d:
    # Filtrar datos para el d actual
    datos_d = datos[datos['d'] == d_val].copy()
    
    # Separar por modalidad
    sin_diff = datos_d[datos_d['Modalidad'] == 'SIN_DIFF']['ECRPS'].values
    con_diff = datos_d[datos_d['Modalidad'] == 'CON_DIFF']['ECRPS'].values
    
    # Verificar que ambas modalidades tengan datos
    if len(sin_diff) == 0 or len(con_diff) == 0:
        print(f"Advertencia: d={d_val} no tiene datos para ambas modalidades")
        continue
    
    # Verificar que tengan la misma longitud
    if len(sin_diff) != len(con_diff):
        print(f"Advertencia: d={d_val} tiene diferente número de observaciones entre modalidades")
        min_len = min(len(sin_diff), len(con_diff))
        sin_diff = sin_diff[:min_len]
        con_diff = con_diff[:min_len]
    
    # Calcular estadísticas descriptivas
    ecrps_sin_diff_mean = np.mean(sin_diff)
    ecrps_con_diff_mean = np.mean(con_diff)
    diferencia = ecrps_sin_diff_mean - ecrps_con_diff_mean
    
    # Realizar test Diebold-Mariano modificado
    hln_dm_stat, p_value, dm_stat = modified_diebold_mariano_test(sin_diff, con_diff, h=1)
    
    # Determinar significancia (niveles comunes: 0.01, 0.05, 0.10)
    if p_value < 0.01:
        significativo = "***"
    elif p_value < 0.05:
        significativo = "**"
    elif p_value < 0.10:
        significativo = "*"
    else:
        significativo = "No"
    
    # Agregar a resultados
    resultados.append({
        'd': d_val,
        'N_obs': len(sin_diff),
        'ECRPS_SIN_DIFF': ecrps_sin_diff_mean,
        'ECRPS_CON_DIFF': ecrps_con_diff_mean,
        'Diferencia': diferencia,
        'DM_stat': dm_stat,
        'HLN-DM_stat': hln_dm_stat,
        'p_valor': p_value,
        'Significativo': significativo
    })

# Crear DataFrame con resultados
resultados_df = pd.DataFrame(resultados)

# Formatear para mejor visualización
resultados_df['ECRPS_SIN_DIFF'] = resultados_df['ECRPS_SIN_DIFF'].round(6)
resultados_df['ECRPS_CON_DIFF'] = resultados_df['ECRPS_CON_DIFF'].round(6)
resultados_df['Diferencia'] = resultados_df['Diferencia'].round(6)
resultados_df['DM_stat'] = resultados_df['DM_stat'].round(4)
resultados_df['HLN-DM_stat'] = resultados_df['HLN-DM_stat'].round(4)
resultados_df['p_valor'] = resultados_df['p_valor'].round(4)

# Mostrar resultados
print("\n" + "="*100)
print("TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación SIN_DIFF vs CON_DIFF por valor de d")
print("="*100)
print("\nH0: No hay diferencia significativa entre las modalidades")
print("H1: Hay diferencia significativa entre las modalidades")
print("\nSignificancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo")
print("\n")
print(resultados_df.to_string(index=False))

# Resumen de resultados
print("\n" + "="*100)
print("RESUMEN")
print("="*100)
n_significativos = len(resultados_df[resultados_df['Significativo'] != 'No'])
n_total = len(resultados_df)
print(f"\nTotal de comparaciones: {n_total}")
print(f"Diferencias significativas: {n_significativos} ({100*n_significativos/n_total:.1f}%)")
print(f"No significativas: {n_total - n_significativos} ({100*(n_total-n_significativos)/n_total:.1f}%)")



TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación SIN_DIFF vs CON_DIFF por valor de d

H0: No hay diferencia significativa entre las modalidades
H1: Hay diferencia significativa entre las modalidades

Significancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo


 d  N_obs  ECRPS_SIN_DIFF  ECRPS_CON_DIFF    Diferencia  DM_stat  HLN-DM_stat  p_valor Significativo
 1   1680    5.495070e-01    5.485800e-01  9.280000e-04   0.8532       0.8529   0.3938            No
 2   1680    2.884661e+01    2.883279e+01  1.382000e-02   2.1456       2.1449   0.0321            **
 3   1680    2.725984e+01    2.726032e+01 -4.790000e-04  -0.0538      -0.0538   0.9571            No
 4   1680    2.704997e+01    2.704544e+01  4.532000e-03   0.3388       0.3387   0.7349            No
 5   1680    3.451632e+01    2.821125e+01  6.305071e+00   5.9399       5.9381   0.0000           ***
 7   1680    6.686329e+04    1.261129e+03  6.560216e+04  24.3102      24.3030   0.0000           ***
10   1680    1.890

In [8]:
import pandas as pd
import numpy as np
from scipy import stats

# Cargar datos
base = pd.read_excel("./datos/Simulacion/Diferenciado/resultados_UNION_ARIMA.xlsx")

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano modificado con corrección Harvey-Leybourne-Newbold (1997)
    
    Parameters:
    -----------
    errors1, errors2 : array-like
        Errores de pronóstico (ECRPS) de los dos modelos
    h : int
        Horizonte de pronóstico (forecast horizon)
    
    Returns:
    --------
    hln_dm_stat : float
        Estadístico DM corregido (HLN-DM)
    p_value : float
        P-valor usando distribución t-Student con T-1 grados de libertad
    dm_stat : float
        Estadístico DM original (sin corrección)
    """
    # Calcular diferencial de pérdida
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    # Calcular autocovarianzas
    def gamma_d(k):
        if k == 0:
            return np.var(d, ddof=1)
        else:
            return np.mean((d[k:] - d_bar) * (d[:-k] - d_bar))
    
    # Estimar la varianza de largo plazo usando Newey-West
    # Para h-step-ahead forecasts, incluimos hasta h-1 lags
    gamma_0 = gamma_d(0)
    gamma_sum = gamma_0
    
    if h > 1:
        for k in range(1, h):
            gamma_k = gamma_d(k)
            gamma_sum += 2 * gamma_k
    
    var_d = gamma_sum / T
    
    if var_d <= 0:
        return 0, 1.0, 0
    
    # Estadístico DM original
    dm_stat = d_bar / np.sqrt(var_d)
    
    # Corrección Harvey-Leybourne-Newbold (1997)
    correction_factor = np.sqrt((T + 1 - 2*h + h*(h-1)) / T)
    hln_dm_stat = correction_factor * dm_stat
    
    # P-valor usando t-Student con T-1 grados de libertad
    df = T - 1
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df))
    
    return hln_dm_stat, p_value, dm_stat

# Separar por diferenciación
sin_diff = base[base['Diferenciacion'] == 'No'].copy()
con_diff = base[base['Diferenciacion'] == 'Si'].copy()

# Métodos a comparar (todos excepto las columnas de configuración)
metodos = ['Block Bootstrapping', 'Sieve Bootstrap', 'LSPM', 'LSPMW', 
           'AREPD', 'MCPS', 'AV-MCPS', 'DeepAR', 'EnCQR-LSTM']

# Lista para almacenar resultados
resultados = []

print("\n" + "="*120)
print("TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación Sin Diferenciación (No) vs Con Diferenciación (Si)")
print("="*120)
print("\nH0: No hay diferencia significativa entre aplicar o no diferenciación")
print("H1: Hay diferencia significativa entre aplicar o no diferenciación")
print("\nSignificancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo")
print("="*120)

# Iterar sobre cada método
for metodo in metodos:
    # Obtener valores de ECRPS para cada modalidad
    ecrps_sin = sin_diff[metodo].values
    ecrps_con = con_diff[metodo].values
    
    # Verificar que ambas modalidades tengan datos
    if len(ecrps_sin) == 0 or len(ecrps_con) == 0:
        print(f"\nAdvertencia: {metodo} no tiene datos para ambas modalidades")
        continue
    
    # Verificar que tengan la misma longitud
    if len(ecrps_sin) != len(ecrps_con):
        print(f"\nAdvertencia: {metodo} tiene diferente número de observaciones")
        min_len = min(len(ecrps_sin), len(ecrps_con))
        ecrps_sin = ecrps_sin[:min_len]
        ecrps_con = ecrps_con[:min_len]
    
    # Calcular estadísticas descriptivas
    ecrps_sin_mean = np.mean(ecrps_sin)
    ecrps_con_mean = np.mean(ecrps_con)
    diferencia = ecrps_sin_mean - ecrps_con_mean
    
    # Realizar test Diebold-Mariano modificado
    hln_dm_stat, p_value, dm_stat = modified_diebold_mariano_test(ecrps_sin, ecrps_con, h=1)
    
    # Determinar significancia
    if p_value < 0.01:
        significativo = "***"
    elif p_value < 0.05:
        significativo = "**"
    elif p_value < 0.10:
        significativo = "*"
    else:
        significativo = "No"
    
    # Agregar a resultados
    resultados.append({
        'Método': metodo,
        'N_obs': len(ecrps_sin),
        'ECRPS_Sin_Diff': ecrps_sin_mean,
        'ECRPS_Con_Diff': ecrps_con_mean,
        'Diferencia': diferencia,
        'DM_stat': dm_stat,
        'HLN-DM_stat': hln_dm_stat,
        'p_valor': p_value,
        'Significativo': significativo
    })

# Crear DataFrame con resultados
resultados_df = pd.DataFrame(resultados)

# Formatear para mejor visualización
resultados_df['ECRPS_Sin_Diff'] = resultados_df['ECRPS_Sin_Diff'].round(6)
resultados_df['ECRPS_Con_Diff'] = resultados_df['ECRPS_Con_Diff'].round(6)
resultados_df['Diferencia'] = resultados_df['Diferencia'].round(6)
resultados_df['DM_stat'] = resultados_df['DM_stat'].round(4)
resultados_df['HLN-DM_stat'] = resultados_df['HLN-DM_stat'].round(4)
resultados_df['p_valor'] = resultados_df['p_valor'].round(4)

# Mostrar resultados
print("\n")
print(resultados_df.to_string(index=False))

# Resumen de resultados
print("\n" + "="*120)
print("RESUMEN")
print("="*120)
n_significativos = len(resultados_df[resultados_df['Significativo'] != 'No'])
n_total = len(resultados_df)
print(f"\nTotal de métodos comparados: {n_total}")
print(f"Diferencias significativas: {n_significativos} ({100*n_significativos/n_total:.1f}%)")
print(f"No significativas: {n_total - n_significativos} ({100*(n_total-n_significativos)/n_total:.1f}%)")

# Mostrar cuáles métodos tienen diferencias significativas
if n_significativos > 0:
    print("\nMétodos con diferencias significativas:")
    metodos_sig = resultados_df[resultados_df['Significativo'] != 'No'][['Método', 'Significativo', 'p_valor']]
    print(metodos_sig.to_string(index=False))


TEST DIEBOLD-MARIANO MODIFICADO (HLN): Comparación Sin Diferenciación (No) vs Con Diferenciación (Si)

H0: No hay diferencia significativa entre aplicar o no diferenciación
H1: Hay diferencia significativa entre aplicar o no diferenciación

Significancia: *** p<0.01, ** p<0.05, * p<0.10, No = no significativo


             Método  N_obs  ECRPS_Sin_Diff  ECRPS_Con_Diff  Diferencia  DM_stat  HLN-DM_stat  p_valor Significativo
Block Bootstrapping   1680       11.251601        0.666133   10.585468  22.5804      22.5736   0.0000           ***
    Sieve Bootstrap   1680        0.547481        0.546020    0.001461   1.7771       1.7766   0.0758             *
               LSPM   1680        1.064804        0.648039    0.416766  24.1811      24.1739   0.0000           ***
              LSPMW   1680        3.079645        0.767172    2.312473  26.6766      26.6686   0.0000           ***
              AREPD   1680       10.031183        0.704149    9.327035  21.1620      21.1557   0.0000     

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings

# Configuración inicial
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'

# ====================================================================================
# 1. PREPARACIÓN DE DIRECTORIOS
# ====================================================================================
output_dir = Path("./Resultados_analisis/Multi_d")
output_dir.mkdir(parents=True, exist_ok=True)
plots_dir = output_dir / "Graficos_Analisis"
plots_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
path_excel = "./datos/Simulacion/Multi_D/resultados_ARIMA_d1_a_d10_DOBLE_MODALIDAD_COMPLETO.xlsx"
try:
    df = pd.read_excel(path_excel)
    print(f"✓ Datos cargados: {df.shape}")
    print(f"✓ Columnas: {df.columns.tolist()}")
except Exception as e:
    print(f"Error crítico: {e}")
    exit()

# Identificar columnas
var_cols = ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 
            'Varianza', 'Modalidad', 'Valor_Observado']
model_cols = [col for col in df.columns if col not in var_cols]
print(f"✓ Modelos identificados: {model_cols}")
print(f"✓ Valores únicos de d: {sorted(df['d'].unique())}")

# ====================================================================================
# TEST DE DIEBOLD-MARIANO CON CORRECCIÓN
# ====================================================================================
def diebold_mariano_test(errors1, errors2, h=1):
    """Test de Diebold-Mariano con corrección Harvey, Leybourne y Newbold"""
    d = errors1**2 - errors2**2
    n = len(d)
    d_mean = np.mean(d)
    gamma_0 = np.var(d, ddof=1)
    dm_stat = d_mean / np.sqrt(gamma_0 / n)
    dm_stat_corrected = dm_stat * np.sqrt((n + 1 - 2*h + h*(h-1)/n) / n)
    p_value = 2 * (1 - stats.t.cdf(np.abs(dm_stat_corrected), df=n-1))
    return dm_stat_corrected, p_value

# ====================================================================================
# NUEVO 1: HEATMAPS ECRPS POR MODALIDAD
# ====================================================================================
def plot_heatmaps_ecrps_por_modalidad():
    print("\n=== NUEVO 1: Heatmaps ECRPS por Modalidad ===")
    d_values = sorted(df['d'].unique())
    
    for modalidad in ['SIN_DIFF', 'CON_DIFF']:
        matrix_data = []
        for model in model_cols:
            row = []
            for d_val in d_values:
                subset = df[(df['d'] == d_val) & (df['Modalidad'] == modalidad)]
                avg_ecrps = subset[model].mean()
                row.append(avg_ecrps)
            matrix_data.append(row)
        
        df_heatmap = pd.DataFrame(matrix_data, index=model_cols, columns=[f'd={d}' for d in d_values])
        
        plt.figure(figsize=(14, 10))
        annot_matrix = df_heatmap.applymap(lambda x: f'{x:.2e}' if abs(x) >= 1000 else f'{x:.2f}')
        sns.heatmap(df_heatmap, annot=annot_matrix, fmt='', cmap='RdYlGn_r', 
                   cbar_kws={'label': 'ECRPS'}, linewidths=0.5, center=df_heatmap.mean().mean())
        plt.title(f'Heatmap ECRPS - Modalidad: {modalidad}', fontweight='bold', fontsize=14)
        plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
        plt.ylabel('Modelo', fontweight='bold')
        plt.tight_layout()
        plt.savefig(plots_dir / f'NUEVO_1_heatmap_ecrps_{modalidad}.png')
        plt.close()
        df_heatmap.to_excel(output_dir / f'NUEVO_1_tabla_ecrps_{modalidad}.xlsx')

# ====================================================================================
# NUEVO 2: SENSIBILIDAD AL INCREMENTO DE d
# ====================================================================================
def plot_sensibilidad_incremento_d():
    print("\n=== NUEVO 2: Sensibilidad al Incremento de d ===")
    d_values = sorted(df['d'].unique())
    sensibilidad = []
    
    for model in model_cols:
        cambios = []
        for i in range(len(d_values) - 1):
            avg_d1 = df[df['d'] == d_values[i]][model].mean()
            avg_d2 = df[df['d'] == d_values[i+1]][model].mean()
            cambios.append(abs(avg_d2 - avg_d1))
        
        sensibilidad_modelo = np.mean(cambios)
        std_entre_d = np.std([df[df['d'] == d][model].mean() for d in d_values])
        sensibilidad.append({'Modelo': model, 'Sensibilidad_Media': sensibilidad_modelo, 
                           'Std_entre_d': std_entre_d})
    
    df_sens = pd.DataFrame(sensibilidad).sort_values('Sensibilidad_Media', ascending=False)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    colors = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(df_sens)))
    
    bars1 = ax1.barh(df_sens['Modelo'], df_sens['Sensibilidad_Media'], color=colors, edgecolor='black')
    ax1.bar_label(bars1, fmt='%.3f', padding=5, fontsize=8)
    ax1.set_xlabel('Sensibilidad Media', fontweight='bold')
    ax1.set_title('Sensibilidad por Cambio Promedio', fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    bars2 = ax2.barh(df_sens['Modelo'], df_sens['Std_entre_d'], color=colors, edgecolor='black')
    ax2.bar_label(bars2, fmt='%.3f', padding=5, fontsize=8)
    ax2.set_xlabel('Desviación Estándar', fontweight='bold')
    ax2.set_title('Sensibilidad por Variabilidad', fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_2_sensibilidad_incremento_d.png')
    plt.close()
    df_sens.to_excel(output_dir / 'NUEVO_2_tabla_sensibilidad.xlsx', index=False)
    
    # Trayectorias
    plt.figure(figsize=(14, 8))
    for model in model_cols:
        trayectoria = [df[df['d'] == d][model].mean() for d in d_values]
        plt.plot(d_values, trayectoria, marker='o', label=model, linewidth=2, alpha=0.7)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold', fontsize=12)
    plt.ylabel('ECRPS Promedio', fontweight='bold', fontsize=12)
    plt.title('Trayectorias de ECRPS según d', fontweight='bold', fontsize=14)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_2_trayectorias_d.png')
    plt.close()

# ====================================================================================
# NUEVO 3: TEST DIEBOLD-MARIANO
# ====================================================================================
def plot_heatmap_diebold_mariano():
    print("\n=== NUEVO 3: Test Diebold-Mariano ===")
    d_values = sorted(df['d'].unique())
    results = []
    
    for model in model_cols:
        for d_val in d_values:
            sin_diff = df[(df['d'] == d_val) & (df['Modalidad'] == 'SIN_DIFF')][model].values
            con_diff = df[(df['d'] == d_val) & (df['Modalidad'] == 'CON_DIFF')][model].values
            
            if len(sin_diff) > 5 and len(con_diff) > 5:
                dm_stat, p_value = diebold_mariano_test(sin_diff, con_diff)
                results.append({'Modelo': model, 'd': d_val, 'DM_stat': dm_stat, 
                              'p_valor': p_value, 'Significativo': 'Sí' if p_value < 0.05 else 'No'})
    
    df_dm = pd.DataFrame(results)
    pivot_pvalor = df_dm.pivot(index='Modelo', columns='d', values='p_valor')
    pivot_dm_stat = df_dm.pivot(index='Modelo', columns='d', values='DM_stat')
    
    # Heatmap p-valores
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot_pvalor, annot=True, fmt='.3f', cmap='RdYlGn', 
               cbar_kws={'label': 'p-valor'}, linewidths=0.5, vmin=0, vmax=0.1, center=0.05)
    plt.title('Test DM: P-valores (SIN_DIFF vs CON_DIFF)', fontweight='bold', fontsize=14)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
    plt.ylabel('Modelo', fontweight='bold')
    plt.figtext(0.5, -0.02, 'Valores < 0.05: diferencias significativas', 
               ha='center', fontsize=10, style='italic')
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_3_heatmap_dm_pvalor.png', bbox_inches='tight')
    plt.close()
    
    # Heatmap estadísticos
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot_dm_stat, annot=True, fmt='.2f', cmap='RdBu_r', 
               cbar_kws={'label': 'Estadístico DM'}, linewidths=0.5, center=0)
    plt.title('Test DM: Estadísticos', fontweight='bold', fontsize=14)
    plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
    plt.ylabel('Modelo', fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / 'NUEVO_3_heatmap_dm_stat.png', bbox_inches='tight')
    plt.close()
    
    df_dm.to_excel(output_dir / 'NUEVO_3_tabla_diebold_mariano.xlsx', index=False)
    pivot_pvalor.to_excel(output_dir / 'NUEVO_3_matriz_pvalores.xlsx')

# ====================================================================================
# MODIFICADO 1.2: HEATMAPS POR d
# ====================================================================================
def plot_heatmaps_mejora_por_d():
    print("\n=== MODIFICADO 1.2: Heatmaps por d ===")
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        mejoras = []
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
            mejoras.append(mejora_pct)
        
        df_mejora = pd.DataFrame([mejoras], columns=model_cols, index=[f'd={d_val}'])
        
        plt.figure(figsize=(16, 3))
        sns.heatmap(df_mejora, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=1)
        plt.title(f'Mejora (CON_DIFF vs SIN_DIFF) - d={d_val}', fontweight='bold')
        plt.xlabel('Modelo', fontweight='bold')
        plt.tight_layout()
        plt.savefig(plots_dir / f'MOD_1.2_mejora_d{d_val}.png')
        plt.close()

# ====================================================================================
# MODIFICADO 3: HEATMAPS POR PROCESO
# ====================================================================================
def plot_heatmaps_mejora_por_proceso():
    print("\n=== MODIFICADO 3: Heatmaps por Proceso ===")
    d_values = sorted(df['d'].unique())
    
    for model in model_cols:
        mejoras_proceso = []
        procesos = sorted(df['Proceso'].unique())
        
        for proceso in procesos:
            mejoras_d = []
            for d_val in d_values:
                subset = df[(df['d'] == d_val) & (df['Proceso'] == proceso)]
                sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
                mejoras_d.append(mejora_pct)
            mejoras_proceso.append(mejoras_d)
        
        df_mejora = pd.DataFrame(mejoras_proceso, index=procesos, 
                                columns=[f'd={d}' for d in d_values])
        
        plt.figure(figsize=(14, max(8, len(procesos) * 0.5)))
        sns.heatmap(df_mejora, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
        plt.title(f'Mejora por Proceso - {model}', fontweight='bold')
        plt.xlabel('Orden de Diferenciación (d)', fontweight='bold')
        plt.ylabel('Proceso ARMA', fontweight='bold')
        plt.tight_layout()
        
        safe_name = model.replace(' ', '_').replace('/', '_')
        plt.savefig(plots_dir / f'MOD_3_proceso_{safe_name}.png')
        plt.close()
        df_mejora.to_excel(output_dir / f'MOD_3_proceso_{safe_name}.xlsx')

# ====================================================================================
# MODIFICADO 4: HEATMAPS POR DISTRIBUCIÓN
# ====================================================================================
def plot_heatmaps_mejora_por_distribucion():
    print("\n=== MODIFICADO 4: Heatmaps por Distribución ===")
    d_values = sorted(df['d'].unique())
    
    for model in model_cols:
        mejoras_dist = []
        distribuciones = sorted(df['Distribución'].unique())
        
        for dist in distribuciones:
            mejoras_d = []
            for d_val in d_values:
                subset = df[(df['d'] == d_val) & (df['Distribución'] == dist)]
                sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
                mejoras_d.append(mejora_pct)
            mejoras_dist.append(mejoras_d)
        
        df_mejora = pd.DataFrame(mejoras_dist, index=distribuciones, 
                                columns=[f'd={d}' for d in d_values])
        
        plt.figure(figsize=(14, max(6, len(distribuciones) * 1.5)))
        sns.heatmap(df_mejora, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
        plt.title(f'Mejora por Distribución - {model}', fontweight='bold')
        plt.xlabel('d', fontweight='bold')
        plt.ylabel('Distribución', fontweight='bold')
        plt.tight_layout()
        
        safe_name = model.replace(' ', '_').replace('/', '_')
        plt.savefig(plots_dir / f'MOD_4_dist_{safe_name}.png')
        plt.close()
        df_mejora.to_excel(output_dir / f'MOD_4_dist_{safe_name}.xlsx')

# ====================================================================================
# MODIFICADO 5: MATRICES VARIANZA
# ====================================================================================
def plot_matrices_mejora_por_varianza():
    print("\n=== MODIFICADO 5: Matrices Varianza ===")
    d_values = sorted(df['d'].unique())
    varianzas = sorted(df['Varianza'].unique())
    
    for model in model_cols:
        mejoras_var = []
        for d_val in d_values:
            mejoras_por_var = []
            for var in varianzas:
                subset = df[(df['d'] == d_val) & (df['Varianza'] == var)]
                sin = subset[subset['Modalidad'] == 'SIN_DIFF'][model].mean()
                con = subset[subset['Modalidad'] == 'CON_DIFF'][model].mean()
                mejora_pct = ((sin - con) / sin) * 100 if sin != 0 else 0
                mejoras_por_var.append(mejora_pct)
            mejoras_var.append(mejoras_por_var)
        
        df_mejora = pd.DataFrame(mejoras_var, index=[f'd={d}' for d in d_values], 
                                columns=[f'Var={v}' for v in varianzas])
        
        plt.figure(figsize=(max(10, len(varianzas) * 2), 10))
        sns.heatmap(df_mejora, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                   cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
        plt.title(f'Mejora por Varianza - {model}', fontweight='bold')
        plt.xlabel('Varianza', fontweight='bold')
        plt.ylabel('d', fontweight='bold')
        plt.tight_layout()
        
        safe_name = model.replace(' ', '_').replace('/', '_')
        plt.savefig(plots_dir / f'MOD_5_var_{safe_name}.png')
        plt.close()
        df_mejora.to_excel(output_dir / f'MOD_5_var_{safe_name}.xlsx')

# ====================================================================================
# MODIFICADO 9: EXCEL DETALLADO CON d
# ====================================================================================
def generate_detailed_excel_with_d():
    print("\n=== MODIFICADO 9: Excel con d ===")
    file_path = output_dir / "MOD_9_Detallado_Con_d.xlsx"
    
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        for model in model_cols:
            temp = df.groupby(['d', 'Proceso', 'ARMA_base', 'Distribución', 
                              'Varianza', 'Modalidad'])[model].mean().reset_index()
            
            detailed = temp.pivot_table(
                index=['d', 'Proceso', 'ARMA_base', 'Distribución', 'Varianza'],
                columns='Modalidad', values=model).reset_index()
            
            detailed.columns.name = None
            detailed = detailed.rename(columns={'SIN_DIFF': 'ECRPS_Sin_Dif', 
                                               'CON_DIFF': 'ECRPS_Con_Dif'})
            
            detailed['Mejora_Absoluta'] = detailed['ECRPS_Sin_Dif'] - detailed['ECRPS_Con_Dif']
            detailed['Mejora_%'] = (detailed['Mejora_Absoluta'] / detailed['ECRPS_Sin_Dif']) * 100
            detailed = detailed.sort_values(by=['d', 'Mejora_%'], ascending=[True, False])
            
            sheet_name = model.replace(' ', '_').replace('/', '_')[:31]
            detailed.to_excel(writer, sheet_name=sheet_name, index=False)
            
            workbook = writer.book
            worksheet = writer.sheets[sheet_name]
            fmt_green = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
            fmt_red = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
            
            col_idx = detailed.columns.get_loc('Mejora_%')
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx, 
                                       {'type': 'cell', 'criteria': '>', 'value': 0, 'format': fmt_green})
            worksheet.conditional_format(1, col_idx, len(detailed), col_idx,
                                       {'type': 'cell', 'criteria': '<', 'value': 0, 'format': fmt_red})

# ====================================================================================
# FUNCIONES ORIGINALES
# ====================================================================================
def plot_heatmap_d_vs_modelo():
    print("\n=== Heatmap d vs Modelo ===")
    results = []
    d_values = sorted(df['d'].unique())
    
    for d_val in d_values:
        subset_d = df[df['d'] == d_val]
        for model in model_cols:
            sin = subset_d[subset_d['Modalidad'] == 'SIN_DIFF'][model].mean()
            con = subset_d[subset_d['Modalidad'] == 'CON_DIFF'][model].mean()
            mejora = ((sin - con) / sin) * 100 if sin != 0 else 0
            results.append({'d': d_val, 'Modelo': model, 'Mejora_%': mejora})
    
    df_mejora = pd.DataFrame(results)
    pivot = df_mejora.pivot(index='Modelo', columns='d', values='Mejora_%')
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(pivot, annot=True, fmt=".1f", cmap="RdYlGn", center=0, 
                cbar_kws={'label': 'Mejora (%)'}, linewidths=0.5)
    plt.title("Mejora: Modelo vs d", fontweight='bold', fontsize=14)
    plt.xlabel("Valor de d", fontweight='bold')
    plt.ylabel("Modelo", fontweight='bold')
    plt.tight_layout()
    plt.savefig(plots_dir / '2_heatmap_d_vs_modelo.png')
    plt.close()
    pivot.to_excel(output_dir / '2_tabla_d_vs_modelo.xlsx')

# ====================================================================================
# EJECUCIÓN PRINCIPAL
# ====================================================================================
if __name__ == "__main__":
    print("\n" + "="*80)
    print("INICIANDO ANÁLISIS MULTI-D COMPLETO")
    print("="*80)
    
    # Nuevas funciones
    plot_heatmaps_ecrps_por_modalidad()
    plot_sensibilidad_incremento_d()
    plot_heatmap_diebold_mariano()
    
    # Modificadas
    plot_heatmaps_mejora_por_d()
    plot_heatmaps_mejora_por_proceso()
    plot_heatmaps_mejora_por_distribucion()
    plot_matrices_mejora_por_varianza()
    generate_detailed_excel_with_d()
    
    # Originales
    plot_heatmap_d_vs_modelo()
    
    print("\n" + "="*80)
    print(f"✓ ANÁLISIS COMPLETADO")
    print(f"✓ Resultados: {output_dir}")
    print(f"✓ Gráficos: {plots_dir}")
    print("="*80)

✓ Datos cargados: (23520, 19)
✓ Columnas: ['Paso', 'Proceso', 'p', 'd', 'q', 'ARMA_base', 'Distribución', 'Varianza', 'Modalidad', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Modelos identificados: ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
✓ Valores únicos de d: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(7), np.int64(10)]

INICIANDO ANÁLISIS MULTI-D COMPLETO

=== NUEVO 1: Heatmaps ECRPS por Modalidad ===

=== NUEVO 2: Sensibilidad al Incremento de d ===

=== NUEVO 3: Test Diebold-Mariano ===

=== MODIFICADO 1.2: Heatmaps por d ===

=== MODIFICADO 3: Heatmaps por Proceso ===

=== MODIFICADO 4: Heatmaps por Distribución ===

=== MODIFICADO 5: Matrices Varianza ===

=== MODIFICADO 9: Excel con d ===

=== Heatmap d vs Modelo ===

✓ ANÁLISIS COMPLETADO
✓ Resultados: Resultados_analisis\Multi_d
✓ Gráficos

## Analisis cambio de entrenamiento

### Pre-procesamiento

In [4]:
import pandas as pd
import numpy as np


# Leer los tres archivos
arma_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_ARMA.xlsx")
arima_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_ARIMA.xlsx")
setar_df = pd.read_excel("./datos/Simulacion/Tamaño/resultados_TAMANOS_CRECIENTES_SETAR.xlsx")


arma_df['ESCENARIO'] = "Lineal Estacionario"
arima_df['ESCENARIO'] = "Lineal No estacionario"
setar_df['ESCENARIO'] = "No lineal Estacionario"

# Filtrar los que no tienen "Promedio" en la columna "Paso"
arma_df = arma_df[arma_df['Paso'] != 'Promedio']
arima_df = arima_df[arima_df['Paso'] != 'Promedio']
setar_df = setar_df[setar_df['Paso'] != 'Promedio']

# Lista de modelos (columnas a promediar)
modelos = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR',
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Crear tabla comparativa
comparacion = []

for modelo in modelos:
    fila = {'Modelo': modelo}
    
    # Calcular promedio para cada escenario (de la columna del modelo)
    arma_promedio = arma_df[modelo].mean() if modelo in arma_df.columns else np.nan
    arima_promedio = arima_df[modelo].mean() if modelo in arima_df.columns else np.nan
    setar_promedio = setar_df[modelo].mean() if modelo in setar_df.columns else np.nan
    
    fila['ARMA'] = arma_promedio
    fila['ARIMA'] = arima_promedio
    fila['SETAR'] = setar_promedio
    
    # Determinar mejor escenario (menor promedio)
    promedios = {
        'ARMA': arma_promedio,
        'ARIMA': arima_promedio,
        'SETAR': setar_promedio
    }
    
    # Filtrar NaN si existen
    promedios_validos = {k: v for k, v in promedios.items() if not pd.isna(v)}
    
    if promedios_validos:
        mejor_escenario = min(promedios_validos, key=promedios_validos.get)
        fila['Mejor_Escenario'] = mejor_escenario
    else:
        fila['Mejor_Escenario'] = 'N/A'
    
    comparacion.append(fila)

# Crear DataFrame con la tabla comparativa
tabla_comparativa = pd.DataFrame(comparacion)

# Redondear valores para mejor visualización
columnas_numericas = ['ARMA', 'ARIMA', 'SETAR']
tabla_comparativa[columnas_numericas] = tabla_comparativa[columnas_numericas].round(4)

# Mostrar tabla comparativa
print("\n" + "="*80)
print("TABLA COMPARATIVA DE MODELOS POR ESCENARIO")
print("(Promedio de amplitud de intervalos de predicción)")
print("="*80)
print(tabla_comparativa.to_string(index=False))
print("="*80 + "\n")


# Procesamiento especial para SETAR
if 'Descripción' in setar_df.columns:
    setar_df = setar_df.drop('Descripción', axis=1)

# Concatenar los tres dataframes
base_consolidada = pd.concat([arma_df, arima_df, setar_df], ignore_index=True)

# Guardar en un archivo Excel
base_consolidada.to_excel("./datos/Simulacion/Tamaño/Base_Tamaño_3_escenarios.xlsx", index=False)

print("\nArchivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!")
print(f"\nTotal de filas: {len(base_consolidada)}")
print(f"- ARMA: {len(arma_df)} filas")
print(f"- ARIMA: {len(arima_df)} filas")
print(f"- SETAR: {len(setar_df)} filas")


TABLA COMPARATIVA DE MODELOS POR ESCENARIO
(Promedio de amplitud de intervalos de predicción)
             Modelo   ARMA   ARIMA  SETAR Mejor_Escenario
              AREPD 0.9345 13.0489 0.6971           SETAR
            AV-MCPS 0.6768  3.5050 0.6531           SETAR
Block Bootstrapping 0.9049 15.1183 0.6318           SETAR
             DeepAR 0.5650  3.6998 0.5845            ARMA
         EnCQR-LSTM 0.9515  5.9330 0.8404           SETAR
               LSPM 0.7689  1.0868 0.6586           SETAR
              LSPMW 0.7931  1.0870 0.6754           SETAR
               MCPS 0.6496  3.2780 0.6325           SETAR
    Sieve Bootstrap 0.5541  0.5583 0.6254            ARMA

Tabla comparativa guardada en 'Tabla_Comparativa_Modelos_tamaño.xlsx'

Archivo 'Base_Tamaño_3_escenarios.xlsx' creado exitosamente!

Total de filas: 126000
- ARMA: 42000 filas
- ARIMA: 42000 filas
- SETAR: 42000 filas


### Analisis general

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Tamaño")
output_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion/Tamaño/Base_Tamaño_3_escenarios.xlsx")

print(f"✓ Columnas cargadas: {df.columns.tolist()}")

# Renombrar escenarios
df['ESCENARIO'] = df['ESCENARIO'].replace({
    "Lineal Estacionario": "Lineal Estacionario (ARMA)",
    "Lineal No estacionario": "Lineal No Estacionario (ARIMA)",
    "No lineal Estacionario": "No lineal Estacionario (SETAR)"
})

# Identificar columnas
var_cols = ['Paso', 'Proceso', 'Tipo_Proceso', 'Distribución', 'Varianza', 
            'N_Train', 'N_Calib', 'N_Total', 'Valor_Observado', 'ESCENARIO']
model_cols = [col for col in df.columns if col not in var_cols]

print("="*80)
print("ANÁLISIS DE IMPACTO DE TAMAÑO DE MUESTRA (N_Train, N_Calib)")
print("="*80)
print(f"\nModelos identificados: {len(model_cols)}")
print(f"N_Train únicos: {sorted(df['N_Train'].unique())}")
print(f"N_Calib únicos: {sorted(df['N_Calib'].unique())}")
print(f"Escenarios: {df['ESCENARIO'].unique()}")

# Crear columna combinada
df['Config_Tamaño'] = df['N_Train'].astype(str) + '-' + df['N_Calib'].astype(str)

# ====================================================================================
# CREAR ESTRUCTURA DE CARPETAS POR N_TRAIN
# ====================================================================================

print("\n" + "="*80)
print("CREANDO ESTRUCTURA DE CARPETAS POR N_TRAIN")
print("="*80)

# Obtener valores únicos de N_Train
n_train_values = sorted(df['N_Train'].unique())

# Crear carpetas por N_Train
train_dirs = {}
for n_train in n_train_values:
    train_dir = output_dir / f"N_Train_{n_train}"
    train_dir.mkdir(parents=True, exist_ok=True)
    
    # Subcarpetas
    (train_dir / "Comparaciones").mkdir(exist_ok=True)
    (train_dir / "Interacciones").mkdir(exist_ok=True)
    (train_dir / "Rankings").mkdir(exist_ok=True)
    (train_dir / "Tests_Estadisticos").mkdir(exist_ok=True)
    
    train_dirs[n_train] = train_dir
    print(f"✓ Carpeta creada: N_Train_{n_train}")

# Carpeta para comparaciones entre N_Train
comparacion_dir = output_dir / "Comparaciones_Entre_N_Train"
comparacion_dir.mkdir(exist_ok=True)

# ====================================================================================
# FUNCIÓN AUXILIAR: Test Diebold-Mariano Modificado
# ====================================================================================

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano modificado con corrección Harvey-Leybourne-Newbold
    Más apropiado para series temporales que ANOVA
    """
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    def gamma_d(k):
        if k == 0:
            return np.var(d, ddof=1)
        else:
            return np.mean((d[k:] - d_bar) * (d[:-k] - d_bar))
    
    gamma_0 = gamma_d(0)
    gamma_sum = gamma_0
    
    if h > 1:
        for k in range(1, h):
            gamma_k = gamma_d(k)
            gamma_sum += 2 * gamma_k
    
    var_d = gamma_sum / T
    
    if var_d <= 0:
        return 0, 1.0, 0
    
    dm_stat = d_bar / np.sqrt(var_d)
    correction_factor = np.sqrt((T + 1 - 2*h + h*(h-1)) / T)
    hln_dm_stat = correction_factor * dm_stat
    
    df_test = T - 1
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df_test))
    
    return hln_dm_stat, p_value, dm_stat

# ====================================================================================
# SECCIÓN 1: ANÁLISIS POR N_TRAIN (SEPARADO)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: ANÁLISIS POR N_TRAIN (GRÁFICAS INDIVIDUALES)")
print("="*80)

def plot_ntrain_comparison_bars(n_train, scenario=None):
    """Gráfica de barras ordenadas por N_Calib para un N_Train específico"""
    
    if scenario:
        data = df[(df['N_Train'] == n_train) & (df['ESCENARIO'] == scenario)].copy()
        suffix = scenario.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Impacto de N_Calib (N_Train={n_train})\n{scenario}'
    else:
        data = df[df['N_Train'] == n_train].copy()
        suffix = 'General'
        title = f'Impacto de N_Calib (N_Train={n_train})\nTodos los Escenarios'
    
    n_calib_values = sorted(data['N_Calib'].unique())
    
    for model in model_cols:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        means = [data[data['N_Calib'] == nc][model].mean() for nc in n_calib_values]
        stds = [data[data['N_Calib'] == nc][model].std() for nc in n_calib_values]
        
        colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(n_calib_values)))
        bars = ax.bar(range(len(n_calib_values)), means, color=colors, 
                     alpha=0.85, edgecolor='black', linewidth=1.5,
                     yerr=stds, capsize=5)
        
        ax.set_xticks(range(len(n_calib_values)))
        ax.set_xticklabels([f'N_Calib={nc}' for nc in n_calib_values], 
                          rotation=45, ha='right')
        ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
        ax.set_xlabel('Tamaño de Muestra de Calibración', fontsize=11, fontweight='bold')
        ax.set_title(f'{title}\nModelo: {model}', fontsize=12, fontweight='bold')
        ax.grid(axis='y', alpha=0.3, linestyle='--')
        
        # Anotar valores
        for bar, mean, std in zip(bars, means, stds):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + std,
                   f'{mean:.4f}\n±{std:.4f}',
                   ha='center', va='bottom', fontsize=8)
        
        plt.tight_layout()
        save_path = train_dirs[n_train] / "Comparaciones" / f'bars_{model}_{suffix}.png'
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
    
    print(f"✓ Gráficas de barras guardadas: N_Train={n_train}, {suffix}")

def plot_ntrain_heatmap_relative_change(n_train, scenario=None):
    """Heatmap de cambios relativos para un N_Train específico"""
    
    if scenario:
        data = df[(df['N_Train'] == n_train) & (df['ESCENARIO'] == scenario)].copy()
        suffix = scenario.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Cambio Relativo vs N_Calib Base (N_Train={n_train})\n{scenario}'
    else:
        data = df[df['N_Train'] == n_train].copy()
        suffix = 'General'
        title = f'Cambio Relativo vs N_Calib Base (N_Train={n_train})\nTodos los Escenarios'
    
    n_calib_values = sorted(data['N_Calib'].unique())
    pivot_data = data.groupby('N_Calib')[model_cols].mean()
    
    if len(n_calib_values) > 1:
        baseline = n_calib_values[0]
        relative_changes = ((pivot_data - pivot_data.loc[baseline]) / 
                           pivot_data.loc[baseline] * 100)
        
        fig, ax = plt.subplots(figsize=(12, 8))
        sns.heatmap(relative_changes.T, annot=True, fmt='.2f', cmap='RdYlGn_r',
                   center=0, ax=ax, cbar_kws={'label': 'Cambio Porcentual (%)'},
                   linewidths=1, linecolor='gray', vmin=-50, vmax=50)
        
        ax.set_title(title, fontsize=12, fontweight='bold', pad=15)
        ax.set_xlabel(f'N_Calib (Base: {baseline})', fontsize=11, fontweight='bold')
        ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
        
        plt.tight_layout()
        save_path = train_dirs[n_train] / "Comparaciones" / f'heatmap_cambios_{suffix}.png'
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Heatmap guardado: N_Train={n_train}, {suffix}")

def plot_ntrain_best_config(n_train, scenario=None):
    """Identificar y graficar mejor N_Calib por modelo para un N_Train"""
    
    if scenario:
        data = df[(df['N_Train'] == n_train) & (df['ESCENARIO'] == scenario)].copy()
        suffix = scenario.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Mejor N_Calib por Modelo (N_Train={n_train})\n{scenario}'
    else:
        data = df[df['N_Train'] == n_train].copy()
        suffix = 'General'
        title = f'Mejor N_Calib por Modelo (N_Train={n_train})\nTodos los Escenarios'
    
    best_configs = {}
    for model in model_cols:
        config_means = data.groupby('N_Calib')[model].mean()
        best_ncalib = config_means.idxmin()
        best_value = config_means.min()
        best_configs[model] = {'N_Calib': best_ncalib, 'ECRPS': best_value}
    
    best_df = pd.DataFrame(best_configs).T
    best_df = best_df.sort_values('ECRPS')
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    n_calib_unique = sorted(data['N_Calib'].unique())
    color_map = {nc: plt.cm.Set3(i/len(n_calib_unique)) 
                 for i, nc in enumerate(n_calib_unique)}
    colors = [color_map[nc] for nc in best_df['N_Calib']]
    
    bars = ax.barh(best_df.index, best_df['ECRPS'], color=colors, 
                  alpha=0.85, edgecolor='black', linewidth=1.5)
    
    for bar, (model, row) in zip(bars, best_df.iterrows()):
        width = bar.get_width()
        ax.text(width + width*0.01, bar.get_y() + bar.get_height()/2.,
               f"{row['ECRPS']:.4f} (N_Calib={row['N_Calib']})", 
               ha='left', va='center', fontsize=9, fontweight='bold')
    
    ax.set_xlabel('ECRPS Promedio (Mejor N_Calib)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Leyenda
    from matplotlib.patches import Rectangle
    legend_elements = [Rectangle((0,0),1,1, facecolor=color_map[nc], 
                                 edgecolor='black', label=f'N_Calib={nc}') 
                      for nc in n_calib_unique]
    ax.legend(handles=legend_elements, title='Mejor N_Calib', 
             loc='lower right', fontsize=9)
    
    plt.tight_layout()
    save_path = train_dirs[n_train] / "Rankings" / f'best_ncalib_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Guardar Excel
    best_df.to_excel(train_dirs[n_train] / "Rankings" / f'best_ncalib_{suffix}.xlsx')
    print(f"✓ Mejor N_Calib guardado: N_Train={n_train}, {suffix}")
    
    return best_df

# Ejecutar análisis por N_Train
for n_train in n_train_values:
    print(f"\nProcesando N_Train = {n_train}")
    
    # General
    plot_ntrain_comparison_bars(n_train, None)
    plot_ntrain_heatmap_relative_change(n_train, None)
    plot_ntrain_best_config(n_train, None)
    
    # Por escenario
    for scenario in df['ESCENARIO'].unique():
        plot_ntrain_comparison_bars(n_train, scenario)
        plot_ntrain_heatmap_relative_change(n_train, scenario)
        plot_ntrain_best_config(n_train, scenario)

# ====================================================================================
# SECCIÓN 2: TESTS ESTADÍSTICOS (DIEBOLD-MARIANO)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: TESTS DE DIEBOLD-MARIANO (MÁS APROPIADO PARA SERIES TEMPORALES)")
print("="*80)

def dm_test_between_configs(n_train, scenario=None):
    """
    Test DM comparando diferentes N_Calib para un N_Train fijo
    Más apropiado que ANOVA para series temporales
    """
    
    if scenario:
        data = df[(df['N_Train'] == n_train) & (df['ESCENARIO'] == scenario)].copy()
        suffix = scenario.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Test Diebold-Mariano: N_Calib (N_Train={n_train})\n{scenario}'
    else:
        data = df[df['N_Train'] == n_train].copy()
        suffix = 'General'
        title = f'Test Diebold-Mariano: N_Calib (N_Train={n_train})\nTodos los Escenarios'
    
    n_calib_values = sorted(data['N_Calib'].unique())
    
    if len(n_calib_values) < 2:
        print(f"  ⚠ Solo hay un N_Calib para N_Train={n_train}, {suffix}")
        return
    
    h_forecast = int(data['Paso'].mean())
    
    # Test DM para cada modelo
    results_summary = []
    
    for model in model_cols:
        # Comparar todas las parejas de N_Calib
        comparisons = []
        
        for i, nc1 in enumerate(n_calib_values):
            for nc2 in n_calib_values[i+1:]:
                errors1 = data[data['N_Calib'] == nc1][model].values
                errors2 = data[data['N_Calib'] == nc2][model].values
                
                min_len = min(len(errors1), len(errors2))
                errors1 = errors1[:min_len]
                errors2 = errors2[:min_len]
                
                hln_dm_stat, p_val, _ = modified_diebold_mariano_test(
                    errors1, errors2, h=h_forecast
                )
                
                mean1 = np.mean(errors1)
                mean2 = np.mean(errors2)
                
                comparisons.append({
                    'N_Calib_1': nc1,
                    'N_Calib_2': nc2,
                    'DM_Stat': hln_dm_stat,
                    'P_Value': p_val,
                    'Mean_1': mean1,
                    'Mean_2': mean2,
                    'Significativo': 'Sí' if p_val < 0.05 else 'No',
                    'Mejor': nc1 if mean1 < mean2 else nc2
                })
        
        comp_df = pd.DataFrame(comparisons)
        sig_count = (comp_df['Significativo'] == 'Sí').sum()
        
        results_summary.append({
            'Modelo': model,
            'Comparaciones_Significativas': f"{sig_count}/{len(comparisons)}",
            'Proporcion_Sig': sig_count / len(comparisons) if len(comparisons) > 0 else 0,
            'Mejor_N_Calib': data.groupby('N_Calib')[model].mean().idxmin(),
            'ECRPS_Mejor': data.groupby('N_Calib')[model].mean().min()
        })
        
        # Guardar comparaciones detalladas
        comp_df.to_excel(
            train_dirs[n_train] / "Tests_Estadisticos" / f'dm_comparaciones_{model}_{suffix}.xlsx',
            index=False
        )
    
    results_df = pd.DataFrame(results_summary)
    results_df = results_df.sort_values('Proporcion_Sig', ascending=False)
    
    # Visualización
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors = plt.cm.RdYlGn(results_df['Proporcion_Sig'])
    bars = ax.barh(results_df['Modelo'], results_df['Proporcion_Sig'], 
                  color=colors, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    for bar, (_, row) in zip(bars, results_df.iterrows()):
        width = bar.get_width()
        ax.text(width + 0.02, bar.get_y() + bar.get_height()/2.,
               f"{row['Comparaciones_Significativas']}\n(Mejor: N_Calib={row['Mejor_N_Calib']})",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    ax.set_xlabel('Proporción de Comparaciones Significativas (DM test, α=0.05)', 
                 fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(title + f'\n(h={h_forecast})', fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    ax.set_xlim(0, 1.1)
    
    plt.tight_layout()
    save_path = train_dirs[n_train] / "Tests_Estadisticos" / f'dm_summary_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Guardar resumen
    results_df.to_excel(
        train_dirs[n_train] / "Tests_Estadisticos" / f'dm_summary_{suffix}.xlsx',
        index=False
    )
    
    print(f"✓ Test DM guardado: N_Train={n_train}, {suffix}")

# Ejecutar tests DM
for n_train in n_train_values:
    print(f"\nTests DM para N_Train = {n_train}")
    
    dm_test_between_configs(n_train, None)
    
    for scenario in df['ESCENARIO'].unique():
        dm_test_between_configs(n_train, scenario)

# ====================================================================================
# SECCIÓN 3: INTERACCIONES (GRÁFICAS INDIVIDUALES)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: ANÁLISIS DE INTERACCIONES")
print("="*80)

def plot_interaction_paso(n_train, model, scenario=None):
    """Interacción N_Calib × Horizonte para un modelo específico"""
    
    if scenario:
        data = df[(df['N_Train'] == n_train) & (df['ESCENARIO'] == scenario)].copy()
        suffix = scenario.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Interacción N_Calib × Horizonte (N_Train={n_train})\n{scenario}\nModelo: {model}'
    else:
        data = df[df['N_Train'] == n_train].copy()
        suffix = 'General'
        title = f'Interacción N_Calib × Horizonte (N_Train={n_train})\nModelo: {model}'
    
    n_calib_values = sorted(data['N_Calib'].unique())
    pasos = sorted(data['Paso'].unique())
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(n_calib_values)))
    
    for idx, nc in enumerate(n_calib_values):
        means = [data[(data['N_Calib'] == nc) & (data['Paso'] == p)][model].mean() 
                for p in pasos]
        ax.plot(pasos, means, marker='o', label=f'N_Calib={nc}',
               color=colors[idx], linewidth=2.5, markersize=8)
    
    ax.set_xlabel('Horizonte de Pronóstico (Paso)', fontsize=11, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(title='Calibración', fontsize=9, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = train_dirs[n_train] / "Interacciones" / f'paso_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

def plot_interaction_varianza(n_train, model, scenario=None):
    """Interacción N_Calib × Varianza para un modelo específico"""
    
    if scenario:
        data = df[(df['N_Train'] == n_train) & (df['ESCENARIO'] == scenario)].copy()
        suffix = scenario.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Interacción N_Calib × Varianza (N_Train={n_train})\n{scenario}\nModelo: {model}'
    else:
        data = df[df['N_Train'] == n_train].copy()
        suffix = 'General'
        title = f'Interacción N_Calib × Varianza (N_Train={n_train})\nModelo: {model}'
    
    n_calib_values = sorted(data['N_Calib'].unique())
    varianzas = sorted(data['Varianza'].unique())
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.plasma(np.linspace(0.2, 0.9, len(n_calib_values)))
    
    for idx, nc in enumerate(n_calib_values):
        means = [data[(data['N_Calib'] == nc) & (data['Varianza'] == v)][model].mean() 
                for v in varianzas]
        ax.plot(varianzas, means, marker='s', label=f'N_Calib={nc}',
               color=colors[idx], linewidth=2.5, markersize=8)
    
    ax.set_xlabel('Varianza del Proceso', fontsize=11, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(title='Calibración', fontsize=9, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = train_dirs[n_train] / "Interacciones" / f'varianza_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

# Ejecutar interacciones
for n_train in n_train_values:
    print(f"\nInteracciones para N_Train = {n_train}")
    
    for model in model_cols:
        # General
        plot_interaction_paso(n_train, model, None)
        plot_interaction_varianza(n_train, model, None)
        
        # Por escenario
        for scenario in df['ESCENARIO'].unique():
            plot_interaction_paso(n_train, model, scenario)
            plot_interaction_varianza(n_train, model, scenario)

# ====================================================================================
# SECCIÓN 4: COMPARACIONES ENTRE N_TRAIN
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: COMPARACIONES ENTRE DIFERENTES N_TRAIN")
print("="*80)

def compare_ntrain_values(model, scenario=None):
    """Compara el efecto de diferentes N_Train para un modelo"""
    
    if scenario:
        data = df[df['ESCENARIO'] == scenario].copy()
        suffix = scenario.replace(" ", "_").replace("(", "").replace(")", "")
        title = f'Comparación Entre N_Train\n{scenario}\nModelo: {model}'
    else:
        data = df.copy()
        suffix = 'General'
        title = f'Comparación Entre N_Train\nModelo: {model}'
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    n_train_vals = sorted(data['N_Train'].unique())
    colors = plt.cm.Set2(np.linspace(0, 1, len(n_train_vals)))
    
    for idx, nt in enumerate(n_train_vals):
        data_nt = data[data['N_Train'] == nt]
        n_calib_vals = sorted(data_nt['N_Calib'].unique())
        
        means = [data_nt[data_nt['N_Calib'] == nc][model].mean() for nc in n_calib_vals]
        
        ax.plot(n_calib_vals, means, marker='o', label=f'N_Train={nt}',
               color=colors[idx], linewidth=2.5, markersize=8)
    
    ax.set_xlabel('N_Calib', fontsize=11, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(title='Entrenamiento', fontsize=9, loc='best')
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = comparacion_dir / f'ntrain_comparison_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

# Ejecutar comparaciones entre N_Train
for model in model_cols:
    compare_ntrain_values(model, None)
    for scenario in df['ESCENARIO'].unique():
        compare_ntrain_values(model, scenario)

print(f"✓ Comparaciones entre N_Train guardadas")



✓ Columnas cargadas: ['Paso', 'Proceso', 'Tipo_Proceso', 'Distribución', 'Varianza', 'N_Train', 'N_Calib', 'N_Total', 'Valor_Observado', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'ESCENARIO']
ANÁLISIS DE IMPACTO DE TAMAÑO DE MUESTRA (N_Train, N_Calib)

Modelos identificados: 9
N_Train únicos: [np.int64(100), np.int64(200), np.int64(300), np.int64(500), np.int64(1000)]
N_Calib únicos: [np.int64(20), np.int64(40), np.int64(60), np.int64(100), np.int64(200)]
Escenarios: ['Lineal Estacionario (ARMA)' 'Lineal No Estacionario (ARIMA)'
 'No lineal Estacionario (SETAR)']

CREANDO ESTRUCTURA DE CARPETAS POR N_TRAIN
✓ Carpeta creada: N_Train_100
✓ Carpeta creada: N_Train_200
✓ Carpeta creada: N_Train_300
✓ Carpeta creada: N_Train_500
✓ Carpeta creada: N_Train_1000

SECCIÓN 1: ANÁLISIS POR N_TRAIN (GRÁFICAS INDIVIDUALES)

Procesando N_Train = 100
✓ Gráficas de barras guardadas: N_Train=100, General
✓ Heatmap guardado: N_Train=10

## Analisis proporciones

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
from pathlib import Path
import itertools

warnings.filterwarnings('ignore')

# Configuración general
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'serif'
sns.set_palette("husl")

# Crear carpeta de resultados
output_dir = Path("./Resultados_analisis/Proporciones")
output_dir.mkdir(parents=True, exist_ok=True)

# Cargar datos
df = pd.read_excel("./datos/Simulacion/proporciones/resultados_PROPORCIONES_240_TODOS.xlsx")

print(f"✓ Columnas cargadas: {df.columns.tolist()}")
print(f"✓ Dimensiones: {df.shape}")

# ====================================================================================
# RENOMBRAR Y PREPARAR DATOS
# ====================================================================================

# Renombrar Tipo_Proceso
tipo_proceso_map = {
    "ARMA": "Lineal Estacionario (ARMA)",
    "ARIMA": "Lineal No Estacionario (ARIMA)",
    "SETAR": "No Lineal Estacionario (SETAR)"
}

df['Tipo_Proceso'] = df['Tipo_Proceso'].replace(tipo_proceso_map)

# Limpiar y convertir Prop_Calib de manera robusta
def clean_prop_calib(value):
    """Limpia y convierte valores de proporción a entero"""
    if pd.isna(value):
        return np.nan
    
    # Convertir a string y limpiar
    value_str = str(value).strip()
    
    # Si tiene %, extraer el número
    if '%' in value_str:
        # Tomar solo hasta el primer %
        value_str = value_str.split('%')[0]
    
    # Convertir a float
    try:
        value_num = float(value_str)
        # Si es decimal (0.10, 0.20), multiplicar por 100
        if value_num < 1:
            return int(value_num * 100)
        # Si ya es porcentaje (10, 20), devolverlo como entero
        else:
            return int(value_num)
    except:
        return np.nan

df['Prop_Calib_Pct'] = df['Prop_Calib'].apply(clean_prop_calib)

# Verificar que no haya NaN
if df['Prop_Calib_Pct'].isna().any():
    print("⚠ ADVERTENCIA: Se encontraron valores NaN en Prop_Calib_Pct")
    print(f"Valores originales problemáticos: {df[df['Prop_Calib_Pct'].isna()]['Prop_Calib'].unique()}")
    # Eliminar filas con NaN
    df = df.dropna(subset=['Prop_Calib_Pct'])

# Actualizar Prop_Calib también como decimal
df['Prop_Calib'] = df['Prop_Calib_Pct'] / 100

# **NUEVO: Limpiar columna Paso**
def clean_paso(value):
    """Limpia y convierte valores de Paso a entero"""
    if pd.isna(value):
        return np.nan
    
    # Convertir a string y limpiar
    value_str = str(value).strip()
    
    # Extraer solo números
    import re
    numbers = re.findall(r'\d+', value_str)
    
    if numbers:
        return int(numbers[0])  # Tomar el primer número encontrado
    else:
        return np.nan

df['Paso'] = df['Paso'].apply(clean_paso)

# Verificar Paso
if df['Paso'].isna().any():
    print("⚠ ADVERTENCIA: Se encontraron valores NaN en Paso")
    df = df.dropna(subset=['Paso'])

# **NUEVO: Limpiar otras columnas numéricas que puedan tener problemas**
numeric_cols = ['N_Train', 'N_Calib', 'Varianza']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Limpiar columnas de modelos (asegurar que son numéricas)
for model in model_cols:
    df[model] = pd.to_numeric(df[model], errors='coerce')

# Eliminar filas con NaN en columnas críticas
critical_cols = ['Paso', 'Prop_Calib_Pct', 'N_Train', 'N_Calib'] + model_cols
df = df.dropna(subset=critical_cols)

print("\n" + "="*80)
print("ANÁLISIS DE IMPACTO DE PROPORCIONES DE CALIBRACIÓN")
print("="*80)
print(f"\nDatos después de limpieza: {df.shape[0]} filas, {df.shape[1]} columnas")
print(f"Tamaño base de datos: 240")
print(f"Proporciones decimales: {sorted(df['Prop_Calib'].unique())}")
print(f"Proporciones (%): {sorted(df['Prop_Calib_Pct'].unique())}")
print(f"Pasos únicos: {sorted(df['Paso'].unique())}")
print(f"N_Train únicos: {sorted(df['N_Train'].unique())}")
print(f"N_Calib únicos: {sorted(df['N_Calib'].unique())}")
print(f"Tipos de proceso: {df['Tipo_Proceso'].unique()}")

# Identificar columnas de modelos
var_cols = ['Paso', 'Proceso', 'Distribución', 'Varianza', 
            'N_Train', 'N_Calib', 'Prop_Calib', 'Prop_Calib_Pct', 'Tipo_Proceso']
model_cols = [col for col in df.columns if col not in var_cols]

print(f"\nModelos identificados ({len(model_cols)}): {model_cols}")

# ====================================================================================
# ESTRUCTURA DE CARPETAS POR PROPORCIÓN
# ====================================================================================

print("\n" + "="*80)
print("CREANDO ESTRUCTURA DE CARPETAS POR PROPORCIÓN")
print("="*80)

prop_values = sorted(df['Prop_Calib_Pct'].unique())
prop_dirs = {}

for prop_pct in prop_values:
    prop_dir = output_dir / f"Proporcion_{prop_pct}pct"
    prop_dir.mkdir(parents=True, exist_ok=True)
    
    # Subcarpetas
    (prop_dir / "Comparaciones_Modelos").mkdir(exist_ok=True)
    (prop_dir / "Tests_DM").mkdir(exist_ok=True)
    (prop_dir / "Interacciones").mkdir(exist_ok=True)
    (prop_dir / "Rankings").mkdir(exist_ok=True)
    
    prop_dirs[prop_pct] = prop_dir
    
    n_calib = df[df['Prop_Calib_Pct'] == prop_pct]['N_Calib'].iloc[0]
    n_train = df[df['Prop_Calib_Pct'] == prop_pct]['N_Train'].iloc[0]
    print(f"✓ Carpeta creada: Proporcion_{prop_pct}% (N_Train={n_train}, N_Calib={n_calib})")

# Carpetas para comparaciones entre proporciones
comparacion_dir = output_dir / "Comparaciones_Entre_Proporciones"
comparacion_dir.mkdir(exist_ok=True)

dm_global_dir = output_dir / "Tests_DM_Global"
dm_global_dir.mkdir(exist_ok=True)

# ====================================================================================
# FUNCIÓN: TEST DIEBOLD-MARIANO
# ====================================================================================

def modified_diebold_mariano_test(errors1, errors2, h=1):
    """
    Test Diebold-Mariano modificado con corrección Harvey-Leybourne-Newbold
    
    Referencias:
    - Diebold & Mariano (1995)
    - Harvey, Leybourne & Newbold (1997)
    """
    d = errors1 - errors2
    d_bar = np.mean(d)
    T = len(d)
    
    def gamma_d(k):
        if k == 0:
            return np.var(d, ddof=1)
        else:
            return np.mean((d[k:] - d_bar) * (d[:-k] - d_bar))
    
    gamma_0 = gamma_d(0)
    gamma_sum = gamma_0
    
    if h > 1:
        for k in range(1, h):
            gamma_k = gamma_d(k)
            gamma_sum += 2 * gamma_k
    
    var_d = gamma_sum / T
    
    if var_d <= 0:
        return 0, 1.0, 0
    
    dm_stat = d_bar / np.sqrt(var_d)
    
    # Corrección Harvey-Leybourne-Newbold para muestras pequeñas
    correction_factor = np.sqrt((T + 1 - 2*h + h*(h-1)) / T)
    hln_dm_stat = correction_factor * dm_stat
    
    df_test = T - 1
    p_value = 2 * (1 - stats.t.cdf(abs(hln_dm_stat), df_test))
    
    return hln_dm_stat, p_value, dm_stat

# ====================================================================================
# SECCIÓN 1: ANÁLISIS DESCRIPTIVO POR PROPORCIÓN
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 1: ANÁLISIS DESCRIPTIVO POR PROPORCIÓN")
print("="*80)

def plot_descriptive_by_proportion(prop_pct, tipo_proceso=None):
    """Análisis descriptivo para una proporción específica"""
    
    if tipo_proceso:
        data = df[(df['Prop_Calib_Pct'] == prop_pct) & 
                  (df['Tipo_Proceso'] == tipo_proceso)].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df[df['Prop_Calib_Pct'] == prop_pct].copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    n_calib = data['N_Calib'].iloc[0]
    n_train = data['N_Train'].iloc[0]
    
    # Calcular estadísticos
    stats_data = []
    for model in model_cols:
        stats_data.append({
            'Modelo': model,
            'Media': data[model].mean(),
            'Mediana': data[model].median(),
            'Desv_Std': data[model].std(),
            'CV': data[model].std() / data[model].mean(),
            'Min': data[model].min(),
            'Max': data[model].max(),
            'Q1': data[model].quantile(0.25),
            'Q3': data[model].quantile(0.75)
        })
    
    stats_df = pd.DataFrame(stats_data).sort_values('Media')
    
    # Gráfica 1: Barras ordenadas con media
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(stats_df)))
    bars = ax.barh(stats_df['Modelo'], stats_df['Media'], 
                  color=colors, alpha=0.85, edgecolor='black', linewidth=1.5,
                  xerr=stats_df['Desv_Std'], capsize=5)
    
    for bar, (_, row) in zip(bars, stats_df.iterrows()):
        width = bar.get_width()
        ax.text(width + width*0.02, bar.get_y() + bar.get_height()/2.,
               f"{row['Media']:.4f}\n±{row['Desv_Std']:.4f}",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    ax.set_xlabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(f'Ranking de Modelos - Proporción {prop_pct}%' + 
                f'{title_suffix}\n(N_Train={n_train}, N_Calib={n_calib})',
                fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = prop_dirs[prop_pct] / "Rankings" / f'ranking_media_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Gráfica 2: Coeficiente de Variación (estabilidad)
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cv_sorted = stats_df.sort_values('CV')
    median_cv = cv_sorted['CV'].median()
    colors_cv = ['#2ecc71' if cv < median_cv else '#e74c3c' for cv in cv_sorted['CV']]
    
    bars = ax.barh(cv_sorted['Modelo'], cv_sorted['CV'], 
                  color=colors_cv, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    ax.axvline(x=median_cv, color='black', linestyle='--', 
              linewidth=2, label=f'Mediana = {median_cv:.4f}')
    
    for bar, (_, row) in zip(bars, cv_sorted.iterrows()):
        width = bar.get_width()
        ax.text(width + 0.005, bar.get_y() + bar.get_height()/2.,
               f"{row['CV']:.4f}",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    ax.set_xlabel('Coeficiente de Variación (Desv.Std / Media)', 
                 fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(f'Estabilidad de Modelos - Proporción {prop_pct}%' + 
                f'{title_suffix}\n(Menor CV = Más Estable)',
                fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = prop_dirs[prop_pct] / "Rankings" / f'estabilidad_cv_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Guardar estadísticos
    stats_df.to_excel(
        prop_dirs[prop_pct] / "Rankings" / f'estadisticos_{suffix}.xlsx',
        index=False
    )
    
    print(f"✓ Análisis descriptivo guardado: Prop={prop_pct}%, {suffix}")
    
    return stats_df

# Ejecutar análisis descriptivo
for prop_pct in prop_values:
    print(f"\nProcesando Proporción {prop_pct}%")
    
    # General
    plot_descriptive_by_proportion(prop_pct, None)
    
    # Por tipo de proceso
    for tipo_proceso in df['Tipo_Proceso'].unique():
        plot_descriptive_by_proportion(prop_pct, tipo_proceso)

# ====================================================================================
# SECCIÓN 2: TESTS DIEBOLD-MARIANO ENTRE MODELOS (MISMA PROPORCIÓN)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 2: TESTS DM ENTRE MODELOS (MISMA PROPORCIÓN)")
print("="*80)

def dm_test_between_models(prop_pct, tipo_proceso=None):
    """
    Test DM comparando todos los modelos para una proporción fija
    Identifica cuál modelo es significativamente mejor
    """
    
    if tipo_proceso:
        data = df[(df['Prop_Calib_Pct'] == prop_pct) & 
                  (df['Tipo_Proceso'] == tipo_proceso)].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df[df['Prop_Calib_Pct'] == prop_pct].copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    n_calib = data['N_Calib'].iloc[0]
    n_train = data['N_Train'].iloc[0]
    h_forecast = int(data['Paso'].mean())
    
    # Matriz de comparaciones
    n_models = len(model_cols)
    n_comparisons = n_models * (n_models - 1) / 2
    alpha = 0.05
    bonferroni_alpha = alpha / n_comparisons
    
    results_matrix = np.zeros((n_models, n_models))
    p_values = np.zeros((n_models, n_models))
    dm_stats = np.zeros((n_models, n_models))
    
    # Extraer datos de modelos
    model_data = {model: data[model].values for model in model_cols}
    
    for i, model1 in enumerate(model_cols):
        for j, model2 in enumerate(model_cols):
            if i == j:
                results_matrix[i, j] = 0
                p_values[i, j] = 1.0
                dm_stats[i, j] = 0
            elif i < j:
                errors1 = model_data[model1]
                errors2 = model_data[model2]
                
                # Asegurar mismo tamaño
                min_len = min(len(errors1), len(errors2))
                errors1 = errors1[:min_len]
                errors2 = errors2[:min_len]
                
                hln_dm_stat, p_val, _ = modified_diebold_mariano_test(
                    errors1, errors2, h=h_forecast
                )
                
                p_values[i, j] = p_val
                p_values[j, i] = p_val
                dm_stats[i, j] = hln_dm_stat
                dm_stats[j, i] = -hln_dm_stat
                
                # Decisión con corrección Bonferroni
                if p_val < bonferroni_alpha:
                    mean1 = np.mean(errors1)
                    mean2 = np.mean(errors2)
                    if mean1 < mean2:
                        results_matrix[i, j] = 1  # Modelo 1 es mejor
                        results_matrix[j, i] = -1
                    else:
                        results_matrix[i, j] = -1
                        results_matrix[j, i] = 1  # Modelo 2 es mejor
                else:
                    results_matrix[i, j] = 0
                    results_matrix[j, i] = 0
    
    # Visualización: Matriz de comparaciones
    fig, ax = plt.subplots(figsize=(16, 14))
    
    cmap = plt.cm.colors.ListedColormap(['#e74c3c', '#fff9c4', '#2ecc71'])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    
    im = ax.imshow(results_matrix, cmap=cmap, norm=norm, aspect='auto')
    
    ax.set_xticks(np.arange(n_models))
    ax.set_yticks(np.arange(n_models))
    ax.set_xticklabels(model_cols, rotation=45, ha='right', fontsize=9)
    ax.set_yticklabels(model_cols, fontsize=9)
    
    # Anotaciones
    for i in range(n_models):
        for j in range(n_models):
            if i == j:
                text = '-'
                color = 'black'
                fontsize = 10
            else:
                val = results_matrix[i, j]
                p_val = p_values[i, j]
                dm_val = dm_stats[i, j]
                
                if val == 1:
                    text = f'✓\nDM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                elif val == -1:
                    text = f'✗\nDM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                else:
                    text = f'≈\np={p_val:.4f}'
                    color = 'black'
                fontsize = 7
                
            ax.text(j, i, text, ha='center', va='center', 
                   color=color, fontsize=fontsize, fontweight='bold')
    
    T = min(len(v) for v in model_data.values())
    df_test = T - 1
    
    ax.set_title(f'Test Diebold-Mariano: Comparación Entre Modelos\n' +
                f'Proporción {prop_pct}% (N_Train={n_train}, N_Calib={n_calib})' +
                title_suffix +
                f'\n(h={h_forecast}, T={T}, df={df_test}, α Bonf={bonferroni_alpha:.5f})',
                fontsize=12, fontweight='bold', pad=20)
    ax.set_xlabel('Modelo (Columna)', fontsize=10, fontweight='bold')
    ax.set_ylabel('Modelo (Fila)', fontsize=10, fontweight='bold')
    
    # Leyenda
    legend_elements = [
        plt.Rectangle((0,0),1,1, facecolor='#2ecc71', 
                     label='Fila supera significativamente a columna'),
        plt.Rectangle((0,0),1,1, facecolor='#e74c3c', 
                     label='Columna supera significativamente a fila'),
        plt.Rectangle((0,0),1,1, facecolor='#fff9c4', 
                     label='Sin diferencia significativa')
    ]
    ax.legend(handles=legend_elements, loc='upper left', 
             bbox_to_anchor=(1.02, 1), fontsize=9)
    
    plt.tight_layout()
    save_path = prop_dirs[prop_pct] / "Tests_DM" / f'dm_matriz_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Resumen: Victorias por modelo
    summary_data = []
    for idx, model in enumerate(model_cols):
        victorias = int(np.sum(results_matrix[idx, :] == 1))
        derrotas = int(np.sum(results_matrix[idx, :] == -1))
        empates = int(np.sum(results_matrix[idx, :] == 0)) - 1
        tasa_victoria = (victorias / (n_models - 1)) * 100
        
        summary_data.append({
            'Modelo': model,
            'Victorias': victorias,
            'Derrotas': derrotas,
            'Empates': empates,
            'Tasa_Victoria_%': tasa_victoria,
            'ECRPS_Medio': data[model].mean()
        })
    
    summary_df = pd.DataFrame(summary_data).sort_values('Tasa_Victoria_%', ascending=False)
    
    # Gráfica: Resumen de victorias
    fig, ax = plt.subplots(figsize=(12, 8))
    
    colors_victory = plt.cm.RdYlGn(summary_df['Tasa_Victoria_%'] / 100)
    bars = ax.barh(summary_df['Modelo'], summary_df['Tasa_Victoria_%'],
                  color=colors_victory, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    for bar, (_, row) in zip(bars, summary_df.iterrows()):
        width = bar.get_width()
        ax.text(width + 2, bar.get_y() + bar.get_height()/2.,
               f"{row['Tasa_Victoria_%']:.1f}%\n({row['Victorias']}/{n_models-1})",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    ax.set_xlabel('Tasa de Victoria (%)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(f'Resumen Test DM: Tasa de Victoria por Modelo\n' +
                f'Proporción {prop_pct}%{title_suffix}',
                fontsize=12, fontweight='bold')
    ax.set_xlim(0, 110)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = prop_dirs[prop_pct] / "Tests_DM" / f'dm_resumen_victorias_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Guardar resultados
    results_df = pd.DataFrame(results_matrix, index=model_cols, columns=model_cols)
    pvalues_df = pd.DataFrame(p_values, index=model_cols, columns=model_cols)
    dmstats_df = pd.DataFrame(dm_stats, index=model_cols, columns=model_cols)
    
    with pd.ExcelWriter(prop_dirs[prop_pct] / "Tests_DM" / f'dm_completo_{suffix}.xlsx') as writer:
        results_df.to_excel(writer, sheet_name='Matriz_Resultados')
        pvalues_df.to_excel(writer, sheet_name='P_valores')
        dmstats_df.to_excel(writer, sheet_name='Estadisticos_DM')
        summary_df.to_excel(writer, sheet_name='Resumen', index=False)
    
    print(f"✓ Test DM entre modelos guardado: Prop={prop_pct}%, {suffix}")
    
    return summary_df

# Ejecutar tests DM entre modelos
for prop_pct in prop_values:
    print(f"\nTests DM para Proporción {prop_pct}%")
    
    dm_test_between_models(prop_pct, None)
    
    for tipo_proceso in df['Tipo_Proceso'].unique():
        dm_test_between_models(prop_pct, tipo_proceso)
# ====================================================================================
# SECCIÓN 3: TESTS DM ENTRE PROPORCIONES (MISMO MODELO)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 3: TESTS DM ENTRE PROPORCIONES (MISMO MODELO)")
print("="*80)

def dm_test_between_proportions(model, tipo_proceso=None):
    """
    Test DM comparando diferentes proporciones para un modelo fijo
    Identifica la proporción óptima para cada modelo
    """
    
    if tipo_proceso:
        data = df[df['Tipo_Proceso'] == tipo_proceso].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df.copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    prop_pcts = sorted(data['Prop_Calib_Pct'].unique())
    h_forecast = int(data['Paso'].mean())
    
    # Matriz de comparaciones
    n_props = len(prop_pcts)
    n_comparisons = n_props * (n_props - 1) / 2
    alpha = 0.05
    bonferroni_alpha = alpha / n_comparisons
    
    results_matrix = np.zeros((n_props, n_props))
    p_values = np.zeros((n_props, n_props))
    dm_stats = np.zeros((n_props, n_props))
    means = np.zeros(n_props)
    
    # Extraer datos por proporción
    prop_data = {}
    for prop_pct in prop_pcts:
        prop_data[prop_pct] = data[data['Prop_Calib_Pct'] == prop_pct][model].values
    
    for i, prop1 in enumerate(prop_pcts):
        means[i] = np.mean(prop_data[prop1])
        
        for j, prop2 in enumerate(prop_pcts):
            if i == j:
                results_matrix[i, j] = 0
                p_values[i, j] = 1.0
                dm_stats[i, j] = 0
            elif i < j:
                errors1 = prop_data[prop1]
                errors2 = prop_data[prop2]
                
                min_len = min(len(errors1), len(errors2))
                errors1 = errors1[:min_len]
                errors2 = errors2[:min_len]
                
                hln_dm_stat, p_val, _ = modified_diebold_mariano_test(
                    errors1, errors2, h=h_forecast
                )
                
                p_values[i, j] = p_val
                p_values[j, i] = p_val
                dm_stats[i, j] = hln_dm_stat
                dm_stats[j, i] = -hln_dm_stat
                
                if p_val < bonferroni_alpha:
                    mean1 = np.mean(errors1)
                    mean2 = np.mean(errors2)
                    if mean1 < mean2:
                        results_matrix[i, j] = 1
                        results_matrix[j, i] = -1
                    else:
                        results_matrix[i, j] = -1
                        results_matrix[j, i] = 1
                else:
                    results_matrix[i, j] = 0
                    results_matrix[j, i] = 0
    
    # Visualización: Matriz
    fig, ax = plt.subplots(figsize=(12, 10))
    
    cmap = plt.cm.colors.ListedColormap(['#e74c3c', '#fff9c4', '#2ecc71'])
    bounds = [-1.5, -0.5, 0.5, 1.5]
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
    
    im = ax.imshow(results_matrix, cmap=cmap, norm=norm, aspect='auto')
    
    prop_labels = [f'{p}%' for p in prop_pcts]
    ax.set_xticks(np.arange(n_props))
    ax.set_yticks(np.arange(n_props))
    ax.set_xticklabels(prop_labels, fontsize=10)
    ax.set_yticklabels(prop_labels, fontsize=10)
    
    for i in range(n_props):
        for j in range(n_props):
            if i == j:
                text = f'-\n{means[i]:.4f}'
                color = 'black'
                fontsize = 9
            else:
                val = results_matrix[i, j]
                p_val = p_values[i, j]
                dm_val = dm_stats[i, j]
                
                if val == 1:
                    text = f'✓\nDM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                elif val == -1:
                    text = f'✗\nDM={dm_val:.2f}\np={p_val:.4f}'
                    color = 'white'
                else:
                    text = f'≈\np={p_val:.4f}'
                    color = 'black'
                fontsize = 8
                
            ax.text(j, i, text, ha='center', va='center', 
                   color=color, fontsize=fontsize, fontweight='bold')
    
    T = min(len(v) for v in prop_data.values())
    df_test = T - 1
    
    ax.set_title(f'Test DM: Comparación Entre Proporciones\n' +
                f'Modelo: {model}{title_suffix}\n' +
                f'(h={h_forecast}, T={T}, df={df_test}, α Bonf={bonferroni_alpha:.5f})',
                fontsize=11, fontweight='bold', pad=20)
    ax.set_xlabel('Proporción de Calibración (%)', fontsize=10, fontweight='bold')
    ax.set_ylabel('Proporción de Calibración (%)', fontsize=10, fontweight='bold')
    
    legend_elements = [
        plt.Rectangle((0,0),1,1, facecolor='#2ecc71', 
                     label='Fila supera significativamente a columna'),
        plt.Rectangle((0,0),1,1, facecolor='#e74c3c', 
                     label='Columna supera significativamente a fila'),
        plt.Rectangle((0,0),1,1, facecolor='#fff9c4',
                     label='Sin diferencia significativa')
    ]
    ax.legend(handles=legend_elements, loc='upper left', 
             bbox_to_anchor=(1.02, 1), fontsize=9)
    
    plt.tight_layout()
    save_path = comparacion_dir / f'dm_proporciones_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Guardar resultados
    results_df = pd.DataFrame(results_matrix, 
                             index=[f'{p}%' for p in prop_pcts], 
                             columns=[f'{p}%' for p in prop_pcts])
    pvalues_df = pd.DataFrame(p_values, 
                             index=[f'{p}%' for p in prop_pcts], 
                             columns=[f'{p}%' for p in prop_pcts])
    dmstats_df = pd.DataFrame(dm_stats, 
                             index=[f'{p}%' for p in prop_pcts], 
                             columns=[f'{p}%' for p in prop_pcts])
    
    with pd.ExcelWriter(comparacion_dir / f'dm_proporciones_completo_{model}_{suffix}.xlsx') as writer:
        results_df.to_excel(writer, sheet_name='Matriz_Resultados')
        pvalues_df.to_excel(writer, sheet_name='P_valores')
        dmstats_df.to_excel(writer, sheet_name='Estadisticos_DM')
    
    # Resumen: Victorias por proporción
    summary_data = []
    for idx, prop in enumerate(prop_pcts):
        victorias = int(np.sum(results_matrix[idx, :] == 1))
        derrotas = int(np.sum(results_matrix[idx, :] == -1))
        empates = int(np.sum(results_matrix[idx, :] == 0)) - 1
        
        summary_data.append({
            'Proporcion_%': prop,
            'Victorias': victorias,
            'Derrotas': derrotas,
            'Empates': empates,
            'ECRPS_Medio': means[idx]
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Gráfica: Resumen victorias
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors_bar = plt.cm.RdYlGn(summary_df['Victorias'] / (n_props - 1))
    bars = ax.bar(range(len(summary_df)), summary_df['Victorias'],
                  color=colors_bar, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    ax.set_xticks(range(len(summary_df)))
    ax.set_xticklabels([f"{p}%" for p in summary_df['Proporcion_%']])
    ax.set_ylabel('Número de Victorias', fontsize=11, fontweight='bold')
    ax.set_xlabel('Proporción de Calibración', fontsize=11, fontweight='bold')
    ax.set_title(f'Resumen Test DM: Victorias por Proporción\n' +
                f'Modelo: {model}{title_suffix}',
                fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    for bar, (_, row) in zip(bars, summary_df.iterrows()):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
               f"{int(row['Victorias'])}/{n_props-1}",
               ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    save_path = comparacion_dir / f'dm_proporciones_resumen_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    summary_df.to_excel(
        comparacion_dir / f'dm_proporciones_resumen_{model}_{suffix}.xlsx',
        index=False
    )
    
    print(f"✓ Test DM entre proporciones guardado: Modelo={model}, {suffix}")
    
    return summary_df

# Ejecutar tests DM entre proporciones
print("\nEjecutando Tests DM entre proporciones...")
for model in model_cols:
    print(f"  Modelo: {model}")
    dm_test_between_proportions(model, None)
    
    for tipo_proceso in df['Tipo_Proceso'].unique():
        dm_test_between_proportions(model, tipo_proceso)

# ====================================================================================
# SECCIÓN 4: COMPARACIONES VISUALES ENTRE PROPORCIONES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 4: COMPARACIONES VISUALES ENTRE PROPORCIONES")
print("="*80)

def plot_evolution_across_proportions(model, tipo_proceso=None):
    """Gráfica de evolución del ECRPS a través de proporciones"""
    
    if tipo_proceso:
        data = df[df['Tipo_Proceso'] == tipo_proceso].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df.copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    prop_pcts = sorted(data['Prop_Calib_Pct'].unique())
    
    means = [data[data['Prop_Calib_Pct'] == p][model].mean() for p in prop_pcts]
    stds = [data[data['Prop_Calib_Pct'] == p][model].std() for p in prop_pcts]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(prop_pcts, means, marker='o', linewidth=2.5, markersize=10,
           color='#3498db', label='Media')
    ax.fill_between(prop_pcts, 
                    np.array(means) - np.array(stds),
                    np.array(means) + np.array(stds),
                    alpha=0.3, color='#3498db', label='±1 Desv.Std')
    
    ax.set_xlabel('Proporción de Calibración (%)', fontsize=11, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_title(f'Evolución del ECRPS: {model}{title_suffix}',
                fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # Anotar valores
    for x, y, std in zip(prop_pcts, means, stds):
        ax.annotate(f'{y:.4f}\n±{std:.4f}',
                   xy=(x, y), xytext=(0, 10),
                   textcoords='offset points', ha='center',
                   fontsize=8, bbox=dict(boxstyle='round,pad=0.3', 
                   facecolor='white', alpha=0.7))
    
    plt.tight_layout()
    save_path = comparacion_dir / f'evolucion_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Evolución guardada: Modelo={model}, {suffix}")

def plot_all_models_comparison(tipo_proceso=None):
    """Comparación de todos los modelos a través de proporciones"""
    
    if tipo_proceso:
        data = df[df['Tipo_Proceso'] == tipo_proceso].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df.copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    prop_pcts = sorted(data['Prop_Calib_Pct'].unique())
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    colors = plt.cm.tab20(np.linspace(0, 1, len(model_cols)))
    
    for idx, model in enumerate(model_cols):
        means = [data[data['Prop_Calib_Pct'] == p][model].mean() for p in prop_pcts]
        ax.plot(prop_pcts, means, marker='o', linewidth=2, markersize=7,
               color=colors[idx], label=model, alpha=0.8)
    
    ax.set_xlabel('Proporción de Calibración (%)', fontsize=11, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_title(f'Comparación de Todos los Modelos{title_suffix}',
                fontsize=12, fontweight='bold')
    ax.legend(fontsize=8, loc='best', ncol=2)
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = comparacion_dir / f'todos_modelos_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Comparación todos los modelos guardada: {suffix}")

# Ejecutar comparaciones visuales
print("\nGenerando comparaciones visuales...")
for model in model_cols:
    plot_evolution_across_proportions(model, None)
    for tipo_proceso in df['Tipo_Proceso'].unique():
        plot_evolution_across_proportions(model, tipo_proceso)

plot_all_models_comparison(None)
for tipo_proceso in df['Tipo_Proceso'].unique():
    plot_all_models_comparison(tipo_proceso)

# ====================================================================================
# SECCIÓN 5: INTERACCIONES ENTRE PROPORCIONES Y OTRAS VARIABLES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 5: ANÁLISIS DE INTERACCIONES")
print("="*80)

def plot_interaction_prop_paso(prop_pct, model, tipo_proceso=None):
    """Interacción Proporción × Horizonte"""
    
    if tipo_proceso:
        data = df[(df['Prop_Calib_Pct'] == prop_pct) & 
                  (df['Tipo_Proceso'] == tipo_proceso)].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df[df['Prop_Calib_Pct'] == prop_pct].copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    pasos = sorted(data['Paso'].unique())
    
    means = [data[data['Paso'] == p][model].mean() for p in pasos]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(pasos, means, marker='o', linewidth=2.5, markersize=10,
           color='#e74c3c')
    
    ax.set_xlabel('Horizonte de Pronóstico (Paso)', fontsize=11, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_title(f'Interacción Horizonte: Proporción {prop_pct}%{title_suffix}\n' +
                f'Modelo: {model}',
                fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = prop_dirs[prop_pct] / "Interacciones" / f'paso_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

def plot_interaction_prop_varianza(prop_pct, model, tipo_proceso=None):
    """Interacción Proporción × Varianza"""
    
    if tipo_proceso:
        data = df[(df['Prop_Calib_Pct'] == prop_pct) & 
                  (df['Tipo_Proceso'] == tipo_proceso)].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df[df['Prop_Calib_Pct'] == prop_pct].copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    varianzas = sorted(data['Varianza'].unique())
    
    means = [data[data['Varianza'] == v][model].mean() for v in varianzas]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(varianzas, means, marker='s', linewidth=2.5, markersize=10,
           color='#9b59b6')
    
    ax.set_xlabel('Varianza del Proceso', fontsize=11, fontweight='bold')
    ax.set_ylabel('ECRPS Promedio', fontsize=11, fontweight='bold')
    ax.set_title(f'Interacción Varianza: Proporción {prop_pct}%{title_suffix}\n' +
                f'Modelo: {model}',
                fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = prop_dirs[prop_pct] / "Interacciones" / f'varianza_{model}_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

# Ejecutar interacciones
print("\nGenerando análisis de interacciones...")
for prop_pct in prop_values:
    print(f"  Proporción {prop_pct}%")
    for model in model_cols:
        plot_interaction_prop_paso(prop_pct, model, None)
        plot_interaction_prop_varianza(prop_pct, model, None)
        
        for tipo_proceso in df['Tipo_Proceso'].unique():
            plot_interaction_prop_paso(prop_pct, model, tipo_proceso)
            plot_interaction_prop_varianza(prop_pct, model, tipo_proceso)

# ====================================================================================
# SECCIÓN 6: HEATMAPS GLOBALES
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6: HEATMAPS GLOBALES")
print("="*80)

def plot_global_heatmap_models_vs_proportions(tipo_proceso=None):
    """Heatmap: Modelos × Proporciones"""
    
    if tipo_proceso:
        data = df[df['Tipo_Proceso'] == tipo_proceso].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df.copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    pivot_data = data.groupby('Prop_Calib_Pct')[model_cols].mean()
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    sns.heatmap(pivot_data.T, annot=True, fmt='.4f', cmap='RdYlGn_r',
               ax=ax, cbar_kws={'label': 'ECRPS Promedio'},
               linewidths=1, linecolor='gray')
    
    ax.set_xlabel('Proporción de Calibración (%)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(f'Heatmap Global: Modelos × Proporciones{title_suffix}',
                fontsize=12, fontweight='bold', pad=15)
    
    # Formato ejes
    ax.set_xticklabels([f'{int(x)}%' for x in pivot_data.index], rotation=0)
    
    plt.tight_layout()
    save_path = comparacion_dir / f'heatmap_global_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Guardar datos
    pivot_data.to_excel(comparacion_dir / f'heatmap_global_data_{suffix}.xlsx')
    
    print(f"✓ Heatmap global guardado: {suffix}")

# Ejecutar heatmaps globales
plot_global_heatmap_models_vs_proportions(None)
for tipo_proceso in df['Tipo_Proceso'].unique():
    plot_global_heatmap_models_vs_proportions(tipo_proceso)

# ====================================================================================
# SECCIÓN 6.5: ANÁLISIS DE SENSIBILIDAD (MODELOS MÁS AFECTADOS POR CAMBIOS)
# ====================================================================================

print("\n" + "="*80)
print("SECCIÓN 6.5: ANÁLISIS DE SENSIBILIDAD")
print("="*80)

# Crear carpeta para análisis de sensibilidad
sensibilidad_dir = output_dir / "Analisis_Sensibilidad"
sensibilidad_dir.mkdir(exist_ok=True)

def analyze_sensitivity_to_proportions(tipo_proceso=None):
    """
    Identifica qué modelos son más sensibles a cambios en la proporción de calibración
    """
    
    if tipo_proceso:
        data = df[df['Tipo_Proceso'] == tipo_proceso].copy()
        suffix = tipo_proceso.replace(" ", "_").replace("(", "").replace(")", "")
        title_suffix = f'\n{tipo_proceso}'
    else:
        data = df.copy()
        suffix = 'General'
        title_suffix = '\nTodos los Procesos'
    
    prop_pcts = sorted(data['Prop_Calib_Pct'].unique())
    
    if len(prop_pcts) < 2:
        print(f"  ⚠ No hay suficientes proporciones para análisis de sensibilidad: {suffix}")
        return
    
    sensitivity_metrics = []
    
    for model in model_cols:
        # Calcular métricas por proporción
        means_by_prop = []
        for prop in prop_pcts:
            mean_val = data[data['Prop_Calib_Pct'] == prop][model].mean()
            means_by_prop.append(mean_val)
        
        means_array = np.array(means_by_prop)
        
        # Métricas de sensibilidad
        baseline = means_array[0]  # Primera proporción como referencia
        
        # 1. Rango de variación absoluta
        rango_absoluto = means_array.max() - means_array.min()
        
        # 2. Rango de variación relativa (%)
        rango_relativo = (rango_absoluto / baseline) * 100
        
        # 3. Desviación estándar de las medias
        volatilidad = np.std(means_array)
        
        # 4. Coeficiente de variación de las medias
        cv_medias = volatilidad / np.mean(means_array)
        
        # 5. Cambio máximo entre proporciones consecutivas
        cambios_consecutivos = np.abs(np.diff(means_array))
        max_cambio_consecutivo = cambios_consecutivos.max()
        max_cambio_consecutivo_pct = (max_cambio_consecutivo / baseline) * 100
        
        # 6. Tendencia (correlación con proporción)
        from scipy.stats import pearsonr, spearmanr
        corr_pearson, p_pearson = pearsonr(prop_pcts, means_array)
        corr_spearman, p_spearman = spearmanr(prop_pcts, means_array)
        
        # 7. Mejor y peor proporción
        best_prop_idx = means_array.argmin()
        worst_prop_idx = means_array.argmax()
        
        sensitivity_metrics.append({
            'Modelo': model,
            'Rango_Absoluto': rango_absoluto,
            'Rango_Relativo_%': rango_relativo,
            'Volatilidad': volatilidad,
            'CV_Medias': cv_medias,
            'Max_Cambio_Consecutivo': max_cambio_consecutivo,
            'Max_Cambio_Consecutivo_%': max_cambio_consecutivo_pct,
            'Correlacion_Pearson': corr_pearson,
            'P_value_Pearson': p_pearson,
            'Correlacion_Spearman': corr_spearman,
            'P_value_Spearman': p_spearman,
            'Mejor_Proporcion_%': prop_pcts[best_prop_idx],
            'Peor_Proporcion_%': prop_pcts[worst_prop_idx],
            'ECRPS_Mejor': means_array[best_prop_idx],
            'ECRPS_Peor': means_array[worst_prop_idx],
            'Media_Global': means_array.mean()
        })
    
    sensitivity_df = pd.DataFrame(sensitivity_metrics)
    
    # ============================================================================
    # GRÁFICA 1: Ranking de Sensibilidad (Rango Relativo)
    # ============================================================================
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    sens_sorted = sensitivity_df.sort_values('Rango_Relativo_%', ascending=False)
    
    colors = plt.cm.RdYlGn_r(sens_sorted['Rango_Relativo_%'] / sens_sorted['Rango_Relativo_%'].max())
    bars = ax.barh(sens_sorted['Modelo'], sens_sorted['Rango_Relativo_%'],
                  color=colors, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    for bar, (_, row) in zip(bars, sens_sorted.iterrows()):
        width = bar.get_width()
        ax.text(width + width*0.02, bar.get_y() + bar.get_height()/2.,
               f"{row['Rango_Relativo_%']:.2f}%\n(Δ={row['Rango_Absoluto']:.4f})",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    ax.set_xlabel('Variación Relativa Máxima (%)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(f'Sensibilidad a Cambios en Proporción de Calibración{title_suffix}\n' +
                '(Mayor valor = Más sensible a cambios)',
                fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Línea de referencia (mediana)
    median_sens = sens_sorted['Rango_Relativo_%'].median()
    ax.axvline(x=median_sens, color='red', linestyle='--', linewidth=2,
              label=f'Mediana = {median_sens:.2f}%')
    ax.legend(fontsize=10)
    
    plt.tight_layout()
    save_path = sensibilidad_dir / f'sensibilidad_ranking_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # ============================================================================
    # GRÁFICA 2: Clasificación por Sensibilidad
    # ============================================================================
    
    # Clasificar modelos
    q33 = sensitivity_df['Rango_Relativo_%'].quantile(0.33)
    q66 = sensitivity_df['Rango_Relativo_%'].quantile(0.66)
    
    def classify_sensitivity(value):
        if value < q33:
            return 'Baja'
        elif value < q66:
            return 'Media'
        else:
            return 'Alta'
    
    sensitivity_df['Clasificacion'] = sensitivity_df['Rango_Relativo_%'].apply(classify_sensitivity)
    
    class_counts = sensitivity_df['Clasificacion'].value_counts()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Subplot 1: Pie chart
    colors_pie = ['#2ecc71', '#f39c12', '#e74c3c']
    explode = (0.05, 0.05, 0.1)
    ax1.pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%',
           colors=colors_pie, explode=explode, startangle=90,
           textprops={'fontsize': 11, 'fontweight': 'bold'})
    ax1.set_title(f'Distribución de Sensibilidad{title_suffix}',
                 fontsize=12, fontweight='bold')
    
    # Subplot 2: Scatter - Sensibilidad vs Performance
    for clase, color, marker in [('Baja', '#2ecc71', 'o'), 
                                  ('Media', '#f39c12', 's'), 
                                  ('Alta', '#e74c3c', '^')]:
        mask = sensitivity_df['Clasificacion'] == clase
        ax2.scatter(sensitivity_df[mask]['Rango_Relativo_%'],
                   sensitivity_df[mask]['Media_Global'],
                   c=color, s=150, alpha=0.7, edgecolors='black',
                   linewidth=1.5, marker=marker, label=clase)
    
    ax2.set_xlabel('Sensibilidad (Variación Relativa %)', fontsize=11, fontweight='bold')
    ax2.set_ylabel('ECRPS Promedio Global', fontsize=11, fontweight='bold')
    ax2.set_title('Sensibilidad vs Performance', fontsize=12, fontweight='bold')
    ax2.legend(title='Clasificación', fontsize=10)
    ax2.grid(True, alpha=0.3, linestyle='--')
    
    # Anotar modelos
    for _, row in sensitivity_df.iterrows():
        ax2.annotate(row['Modelo'], 
                    xy=(row['Rango_Relativo_%'], row['Media_Global']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=7, alpha=0.7)
    
    plt.tight_layout()
    save_path = sensibilidad_dir / f'sensibilidad_clasificacion_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # ============================================================================
    # GRÁFICA 3: Correlación con Proporción (Tendencia)
    # ============================================================================
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    corr_sorted = sensitivity_df.sort_values('Correlacion_Spearman')
    
    # Colores según dirección de correlación
    colors_corr = ['#e74c3c' if x < 0 else '#2ecc71' 
                   for x in corr_sorted['Correlacion_Spearman']]
    
    bars = ax.barh(corr_sorted['Modelo'], corr_sorted['Correlacion_Spearman'],
                  color=colors_corr, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    for bar, (_, row) in zip(bars, corr_sorted.iterrows()):
        width = bar.get_width()
        x_pos = width + (0.02 if width > 0 else -0.02)
        ha = 'left' if width > 0 else 'right'
        
        # Significancia
        sig = '***' if row['P_value_Spearman'] < 0.001 else \
              '**' if row['P_value_Spearman'] < 0.01 else \
              '*' if row['P_value_Spearman'] < 0.05 else 'ns'
        
        ax.text(x_pos, bar.get_y() + bar.get_height()/2.,
               f"{row['Correlacion_Spearman']:.3f} {sig}",
               ha=ha, va='center', fontsize=8, fontweight='bold')
    
    ax.axvline(x=0, color='black', linewidth=2)
    ax.set_xlabel('Correlación de Spearman con Proporción', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(f'Tendencia: Correlación con Proporción de Calibración{title_suffix}\n' +
                '(Negativo = mejora con más calibración, Positivo = empeora)',
                fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    ax.set_xlim(-1.1, 1.1)
    
    # Leyenda de significancia
    ax.text(0.02, 0.98, '*** p<0.001, ** p<0.01, * p<0.05, ns p≥0.05',
           transform=ax.transAxes, fontsize=9, verticalalignment='top',
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    save_path = sensibilidad_dir / f'sensibilidad_correlacion_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # ============================================================================
    # GRÁFICA 4: Cambios Consecutivos Máximos
    # ============================================================================
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    cambio_sorted = sensitivity_df.sort_values('Max_Cambio_Consecutivo_%', ascending=False)
    
    colors_cambio = plt.cm.Reds(cambio_sorted['Max_Cambio_Consecutivo_%'] / 
                                 cambio_sorted['Max_Cambio_Consecutivo_%'].max())
    
    bars = ax.barh(cambio_sorted['Modelo'], cambio_sorted['Max_Cambio_Consecutivo_%'],
                  color=colors_cambio, alpha=0.85, edgecolor='black', linewidth=1.5)
    
    for bar, (_, row) in zip(bars, cambio_sorted.iterrows()):
        width = bar.get_width()
        ax.text(width + width*0.02, bar.get_y() + bar.get_height()/2.,
               f"{row['Max_Cambio_Consecutivo_%']:.2f}%",
               ha='left', va='center', fontsize=8, fontweight='bold')
    
    ax.set_xlabel('Máximo Cambio Entre Proporciones Consecutivas (%)', 
                 fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    ax.set_title(f'Volatilidad: Máximo Cambio Consecutivo{title_suffix}\n' +
                '(Sensibilidad a cambios incrementales)',
                fontsize=12, fontweight='bold')
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    save_path = sensibilidad_dir / f'sensibilidad_cambios_consecutivos_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # ============================================================================
    # GRÁFICA 5: Heatmap de Todas las Métricas de Sensibilidad
    # ============================================================================
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Seleccionar métricas para el heatmap
    metrics_for_heatmap = ['Rango_Relativo_%', 'Volatilidad', 'CV_Medias', 
                          'Max_Cambio_Consecutivo_%', 'Correlacion_Spearman']
    
    heatmap_data = sensitivity_df.set_index('Modelo')[metrics_for_heatmap]
    
    # Normalizar para visualización
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    heatmap_normalized = pd.DataFrame(
        scaler.fit_transform(heatmap_data),
        index=heatmap_data.index,
        columns=heatmap_data.columns
    )
    
    sns.heatmap(heatmap_normalized, annot=False, cmap='RdYlGn_r', center=0,
               ax=ax, cbar_kws={'label': 'Z-score (normalizado)'},
               linewidths=0.5, linecolor='gray')
    
    ax.set_title(f'Perfil de Sensibilidad por Modelo{title_suffix}\n' +
                '(Valores normalizados - Rojo = Alta sensibilidad)',
                fontsize=12, fontweight='bold', pad=15)
    ax.set_xlabel('Métrica de Sensibilidad', fontsize=11, fontweight='bold')
    ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
    
    # Renombrar columnas para mejor visualización
    ax.set_xticklabels(['Rango\nRelativo', 'Volatilidad', 'CV\nMedias', 
                       'Max Cambio\nConsecutivo', 'Correlación\nSpearman'],
                      rotation=45, ha='right')
    
    plt.tight_layout()
    save_path = sensibilidad_dir / f'sensibilidad_heatmap_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # ============================================================================
    # GRÁFICA 6: Top Modelos Más y Menos Sensibles
    # ============================================================================
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    top_sensibles = sensitivity_df.nlargest(5, 'Rango_Relativo_%')
    top_robustos = sensitivity_df.nsmallest(5, 'Rango_Relativo_%')
    
    # Más sensibles
    bars1 = ax1.barh(top_sensibles['Modelo'], top_sensibles['Rango_Relativo_%'],
                    color='#e74c3c', alpha=0.85, edgecolor='black', linewidth=1.5)
    ax1.set_xlabel('Variación Relativa (%)', fontsize=11, fontweight='bold')
    ax1.set_title('Top 5 Modelos MÁS SENSIBLES', fontsize=12, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3, linestyle='--')
    
    for bar, (_, row) in zip(bars1, top_sensibles.iterrows()):
        width = bar.get_width()
        ax1.text(width + 0.5, bar.get_y() + bar.get_height()/2.,
                f"{row['Rango_Relativo_%']:.2f}%",
                ha='left', va='center', fontsize=9, fontweight='bold')
    
    # Menos sensibles (más robustos)
    bars2 = ax2.barh(top_robustos['Modelo'], top_robustos['Rango_Relativo_%'],
                    color='#2ecc71', alpha=0.85, edgecolor='black', linewidth=1.5)
    ax2.set_xlabel('Variación Relativa (%)', fontsize=11, fontweight='bold')
    ax2.set_title('Top 5 Modelos MENOS SENSIBLES (Más Robustos)', 
                 fontsize=12, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3, linestyle='--')
    
    for bar, (_, row) in zip(bars2, top_robustos.iterrows()):
        width = bar.get_width()
        ax2.text(width + 0.5, bar.get_y() + bar.get_height()/2.,
                f"{row['Rango_Relativo_%']:.2f}%",
                ha='left', va='center', fontsize=9, fontweight='bold')
    
    plt.suptitle(f'Comparación de Sensibilidad{title_suffix}',
                fontsize=13, fontweight='bold', y=1.02)
    plt.tight_layout()
    save_path = sensibilidad_dir / f'sensibilidad_top_comparacion_{suffix}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    
    # Guardar tabla completa
    sensitivity_df.to_excel(
        sensibilidad_dir / f'sensibilidad_metricas_{suffix}.xlsx',
        index=False
    )
    
    print(f"✓ Análisis de sensibilidad guardado: {suffix}")
    print(f"  - Modelo más sensible: {sensitivity_df.loc[sensitivity_df['Rango_Relativo_%'].idxmax(), 'Modelo']} " +
          f"({sensitivity_df['Rango_Relativo_%'].max():.2f}%)")
    print(f"  - Modelo más robusto: {sensitivity_df.loc[sensitivity_df['Rango_Relativo_%'].idxmin(), 'Modelo']} " +
          f"({sensitivity_df['Rango_Relativo_%'].min():.2f}%)")
    
    return sensitivity_df

# Ejecutar análisis de sensibilidad
print("\nEjecutando análisis de sensibilidad...")
sensitivity_results = {}

sensitivity_results['General'] = analyze_sensitivity_to_proportions(None)

for tipo_proceso in df['Tipo_Proceso'].unique():
    sensitivity_results[tipo_proceso] = analyze_sensitivity_to_proportions(tipo_proceso)

# Resumen comparativo entre tipos de proceso
print("\n" + "="*80)
print("RESUMEN COMPARATIVO DE SENSIBILIDAD POR TIPO DE PROCESO")
print("="*80)

for tipo_proceso, sens_df in sensitivity_results.items():
    if sens_df is not None and len(sens_df) > 0:
        print(f"\n{tipo_proceso}:")
        print(f"  Sensibilidad promedio: {sens_df['Rango_Relativo_%'].mean():.2f}%")
        print(f"  Modelo más sensible: {sens_df.loc[sens_df['Rango_Relativo_%'].idxmax(), 'Modelo']} " +
              f"({sens_df['Rango_Relativo_%'].max():.2f}%)")
        print(f"  Modelo más robusto: {sens_df.loc[sens_df['Rango_Relativo_%'].idxmin(), 'Modelo']} " +
              f"({sens_df['Rango_Relativo_%'].min():.2f}%)")

print("\n✓ Análisis de sensibilidad completo")

# ====================================================================================
# RESUMEN FINAL Y ESTADÍSTICAS GLOBALES
# ====================================================================================

print("\n" + "="*80)
print("GENERANDO RESUMEN FINAL")
print("="*80)

# Ranking global de modelos (promediando todas las proporciones)
ranking_global = []
for model in model_cols:
    ranking_global.append({
        'Modelo': model,
        'ECRPS_Promedio_Global': df[model].mean(),
        'Desv_Std_Global': df[model].std(),
        'CV_Global': df[model].std() / df[model].mean(),
        'Min_Global': df[model].min(),
        'Max_Global': df[model].max()
    })

ranking_df = pd.DataFrame(ranking_global).sort_values('ECRPS_Promedio_Global')

# Visualización ranking global
fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(ranking_df)))
bars = ax.barh(ranking_df['Modelo'], ranking_df['ECRPS_Promedio_Global'],
              color=colors, alpha=0.85, edgecolor='black', linewidth=1.5,
              xerr=ranking_df['Desv_Std_Global'], capsize=5)

for bar, (_, row) in zip(bars, ranking_df.iterrows()):
    width = bar.get_width()
    ax.text(width + width*0.02, bar.get_y() + bar.get_height()/2.,
           f"{row['ECRPS_Promedio_Global']:.4f}",
           ha='left', va='center', fontsize=9, fontweight='bold')

ax.set_xlabel('ECRPS Promedio Global', fontsize=11, fontweight='bold')
ax.set_ylabel('Modelo', fontsize=11, fontweight='bold')
ax.set_title('Ranking Global de Modelos\n(Promedio de Todas las Proporciones)',
            fontsize=12, fontweight='bold')
ax.grid(axis='x', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig(output_dir / 'ranking_global_final.png', bbox_inches='tight')
plt.close()

# Guardar ranking global
ranking_df.to_excel(output_dir / 'ranking_global_final.xlsx', index=False)

print("\n" + "="*80)
print("✓✓✓ ANÁLISIS COMPLETO FINALIZADO ✓✓✓")
print("="*80)
print(f"\nResultados guardados en: {output_dir}")
print(f"Total de imágenes generadas: {sum([len(list(p.rglob('*.png'))) for p in [output_dir]])}")
print(f"Total de archivos Excel generados: {sum([len(list(p.rglob('*.xlsx'))) for p in [output_dir]])}")
print("\nEstructura de carpetas:")
print(f"  - {len(prop_dirs)} carpetas por proporción")
print(f"  - Comparaciones entre proporciones")
print(f"  - Tests DM globales")
print(f"  - Resumen final")

✓ Columnas cargadas: ['Paso', 'Proceso', 'Distribución', 'Varianza', 'N_Train', 'N_Calib', 'Prop_Calib', 'Block Bootstrapping', 'Sieve Bootstrap', 'LSPM', 'LSPMW', 'AREPD', 'MCPS', 'AV-MCPS', 'DeepAR', 'EnCQR-LSTM', 'Tipo_Proceso']
✓ Dimensiones: (7800, 17)
⚠ ADVERTENCIA: Se encontraron valores NaN en Paso

ANÁLISIS DE IMPACTO DE PROPORCIONES DE CALIBRACIÓN

Datos después de limpieza: 7200 filas, 18 columnas
Tamaño base de datos: 240
Proporciones decimales: [np.float64(0.1), np.float64(0.2), np.float64(0.3), np.float64(0.4), np.float64(0.5)]
Proporciones (%): [np.int64(10), np.int64(20), np.int64(30), np.int64(40), np.int64(50)]
Pasos únicos: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0), np.float64(11.0), np.float64(12.0)]
N_Train únicos: [np.int64(120), np.int64(144), np.int64(168), np.int64(192), np.int64(216)]
N_Calib únicos: [np.int64(24), np.int64(48), np.i

# Analisis Aplicaciones